In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage, AnnotationBbox)
from docxtpl import DocxTemplate, InlineImage
import jinja2
import sys
import os
import time
from docx2pdf import convert
from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename
import re

In [2]:
# local_time = pytz.timezone ("America/Los_Angeles")

In [3]:
Tk().withdraw()
data_file = askopenfilename()
car1_mse_data = askopenfilename()
car2_mse_data = askopenfilename()
car3_mse_data = askopenfilename()

In [4]:
# VBox Data import
if pd.isnull(car1_mse_data):
    c1_df = pd.DataFrame(columns = ['Run', 'Time (s)', 'Speed (Avg) (mph)', 'Speed (Std Dev) (mph)',
       'UTC time (At Start)', 'car'])
elif car1_mse_data == '':
    c1_df = pd.DataFrame(columns = ['Run', 'Time (s)', 'Speed (Avg) (mph)', 'Speed (Std Dev) (mph)',
       'UTC time (At Start)', 'car'])
else:
    c1_df = pd.read_csv(car1_mse_data, skiprows=[0])
    c1_df['car'] = 1

if pd.isnull(car2_mse_data):
    c2_df = pd.DataFrame(columns = ['Run', 'Time (s)', 'Speed (Avg) (mph)', 'Speed (Std Dev) (mph)',
       'UTC time (At Start)', 'car'])
elif car2_mse_data == '':
    c2_df = pd.DataFrame(columns = ['Run', 'Time (s)', 'Speed (Avg) (mph)', 'Speed (Std Dev) (mph)',
       'UTC time (At Start)', 'car'])
else:
    c2_df = pd.read_csv(car2_mse_data, skiprows=[0])
    c2_df['car'] = 2

if pd.isnull(car3_mse_data):
    c3_df = pd.DataFrame(columns = ['Run', 'Time (s)', 'Speed (Avg) (mph)', 'Speed (Std Dev) (mph)',
       'UTC time (At Start)', 'car'])
elif car3_mse_data == '':
    c3_df = pd.DataFrame(columns = ['Run', 'Time (s)', 'Speed (Avg) (mph)', 'Speed (Std Dev) (mph)',
       'UTC time (At Start)', 'car'])
else:
    c3_df = pd.read_csv(car3_mse_data, skiprows=[0])
    c3_df['car'] = 3

In [5]:
c1_df_to_drop = c1_df[~c1_df['Run'].str.isdigit()].index.tolist()
# c1_df.iloc[c1_df_to_drop].tail(50)
c1_df.drop(c1_df.index[c1_df_to_drop], inplace=True)

In [6]:
c3_df_to_drop = c3_df[~c3_df['Run'].str.isdigit()].index.tolist()
# c1_df.iloc[c1_df_to_drop].tail(50)
c3_df.drop(c3_df.index[c3_df_to_drop], inplace=True)
# c3_df.reset_index(drop=True, inplace=True)

In [7]:
c3_df

,Run,Time (s),Speed (Avg) (mph),Speed (Std Dev) (mph),UTC time (At Start),car


In [8]:
# Set size of display
# pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [9]:
c2_df

,Run,Time (s),Speed (Avg) (mph),Speed (Std Dev) (mph),UTC time (At Start),car


In [10]:
# Course data import from excel file
students = pd.read_excel(data_file, sheet_name=("Students"))
raw_runs = pd.read_excel(data_file, sheet_name=("Skill Building")) #Skill building 
raw_values = pd.read_excel(data_file, sheet_name=("Course Values"), skiprows=1)
finalx_df = pd.read_excel(data_file, sheet_name=("Final Exercise")) 
comments_x = pd.read_excel(data_file, sheet_name=('Instructor Comments')).dropna() #.set_index('participant', inplace=True)

/usr/local/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [11]:
raw_values

,Date,Location,Program,Client,Country,Units,Car,Make,LatAcc,Exercise,Chord,MO,Ideal_Time Sec,Cone Penalty Sec,Door Penalty Sec
0,2022-06-30,OFF ROAD Training Center,Habilidades de Manejo Avanzadas [Anti-Secuestro],BASF,MX,KPH,1,VW Jetta 2.5,0.93,Slalom,50.0,3.625,69,2,5
1,NaT,NaN,NaN,NaN,NaN,NaN,2,VW Jetta 2.5,0.93,Lane Change,100.0,13.640,Exercise,Chord,MO
2,NaT,NaN,NaN,NaN,NaN,NaN,3,VW Jetta 2.5,0.93,NaN,NaN,NaN,Slalom,50,3.625
3,NaT,NaN,NaN,NaN,NaN,NaN,4,VW Jetta 2.5,0.93,NaN,NaN,NaN,Lane Change,100,13.64


In [12]:
students.columns = students.columns.str.strip()
students

,ID,Name,Last Name,Company,Gender,Birthday,Fullname
0,NaN,Juan,Guerrero Ortiz,Basf,M,1957-07-31,Juan Guerrero Ortiz
1,NaN,German Enoc,Velazco Gallardo,Basf,M,1977-03-07,German Enoc Velazco Gallardo
2,NaN,Miguel Angel,Ortiz Herrera,Basf,M,1977-09-29,Miguel Angel Ortiz Herrera
3,NaN,Jesús,Cruz González,Basf,M,1964-11-14,Jesús Cruz González
4,NaN,Luis Alberto,Zozaya Ordoñez,Basf,M,1993-01-05,Luis Alberto Zozaya Ordoñez
...,...,...,...,...,...,...,...
213,NaN,NaN,NaN,NaN,NaN,NaT,
214,NaN,NaN,NaN,NaN,NaN,NaT,
215,NaN,NaN,NaN,NaN,NaN,NaT,
216,NaN,NaN,NaN,NaN,NaN,NaT,


In [13]:
finalx_df.head()

,ID,Exercise,Participant,Pressure,Car #,Cones,Doors,Time,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,1,Final Exercise,Demo,0.0,1.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Final Exercise,Demo,0.0,1.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Final Exercise,Demo,0.0,1.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Final Exercise,Miguel Angel Ortiz Herrera,0.0,1.0,1.0,NaN,70.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Final Exercise,Luis Alberto Zozaya Ordoñez,0.0,1.0,NaN,NaN,76.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
finalx_df_empty_rows = finalx_df[finalx_df['Participant'].isnull()].index.tolist()
finalx_df.drop(finalx_df.index[finalx_df_empty_rows], inplace=True)

In [15]:
comments_x.sort_values('Student', ignore_index=True, inplace=True)
comments = pd.Series(comments_x['Comment']).astype(str)

In [16]:
comments_x.rename(columns={'Student':'fullname'}, inplace=True)

In [17]:
# comments_x.set_index('participant', inplace=True)

In [18]:
# students['comment'] = comments['comment']

In [19]:
# comments
raw_values
# raw_runs

,Date,Location,Program,Client,Country,Units,Car,Make,LatAcc,Exercise,Chord,MO,Ideal_Time Sec,Cone Penalty Sec,Door Penalty Sec
0,2022-06-30,OFF ROAD Training Center,Habilidades de Manejo Avanzadas [Anti-Secuestro],BASF,MX,KPH,1,VW Jetta 2.5,0.93,Slalom,50.0,3.625,69,2,5
1,NaT,NaN,NaN,NaN,NaN,NaN,2,VW Jetta 2.5,0.93,Lane Change,100.0,13.640,Exercise,Chord,MO
2,NaT,NaN,NaN,NaN,NaN,NaN,3,VW Jetta 2.5,0.93,NaN,NaN,NaN,Slalom,50,3.625
3,NaT,NaN,NaN,NaN,NaN,NaN,4,VW Jetta 2.5,0.93,NaN,NaN,NaN,Lane Change,100,13.64


In [20]:
raw_values['Date'] = pd.to_datetime(raw_values['Date'])

In [21]:
raw_values.loc[0, 'Ideal_Time Sec'] = pd.to_timedelta(raw_values.loc[0, 'Ideal_Time Sec'], unit='s')

In [22]:
students

,ID,Name,Last Name,Company,Gender,Birthday,Fullname
0,NaN,Juan,Guerrero Ortiz,Basf,M,1957-07-31,Juan Guerrero Ortiz
1,NaN,German Enoc,Velazco Gallardo,Basf,M,1977-03-07,German Enoc Velazco Gallardo
2,NaN,Miguel Angel,Ortiz Herrera,Basf,M,1977-09-29,Miguel Angel Ortiz Herrera
3,NaN,Jesús,Cruz González,Basf,M,1964-11-14,Jesús Cruz González
4,NaN,Luis Alberto,Zozaya Ordoñez,Basf,M,1993-01-05,Luis Alberto Zozaya Ordoñez
...,...,...,...,...,...,...,...
213,NaN,NaN,NaN,NaN,NaN,NaT,
214,NaN,NaN,NaN,NaN,NaN,NaT,
215,NaN,NaN,NaN,NaN,NaN,NaT,
216,NaN,NaN,NaN,NaN,NaN,NaT,


In [23]:
students.rename(columns={
    "Name": "firstname",
    "Last Name": "lastname",
    "Gender": "gender",
    "Birthday": "dob"
}, inplace=True)

In [24]:
students_df = pd.DataFrame(students, columns=["firstname", "lastname", "gender", "dob"])

In [25]:
exercise_values = pd.DataFrame(raw_values, columns=["Exercise", "Chord", "MO",])
exercise_values.dropna(inplace=True)

In [26]:
exercise_values.set_index('Exercise', inplace=True)

In [27]:
exercise_values

,Chord,MO
Exercise,,
Slalom,50.0,3.625
Lane Change,100.0,13.640


In [28]:
rds = []

In [29]:
def radius(data):
    for index, row in data.iterrows():
        x = row['Chord']
        y = row['MO']
        rd = (x**2) / (8*y) + (y/2)
        rds.append(rd)

In [30]:
radius(exercise_values)

In [31]:
exercise_values['Radius'] = rds

In [32]:
exercise_values

,Chord,MO,Radius
Exercise,,,
Slalom,50.0,3.625,88.019397
Lane Change,100.0,13.640,98.462229


In [33]:
course_cars = pd.DataFrame(raw_values, columns=["Car", "Make", "LatAcc",],)
course_cars.set_index('Car', inplace=True)

In [34]:
course_df = pd.DataFrame(
    raw_runs,
    columns=[
        "ID",
        "Car #",
        "Participant",
        "Exercise",
        "Speed Req",
        "V1",
        "V2",
        "V3",
        "Cones",
    ],
).replace('NaN',np.nan)

In [35]:
all_empty_row = course_df[course_df[['Speed Req', 'V1', 'V2', 'V3']].isnull().all(axis=1) == True].index.tolist()
course_df.drop(course_df.index[all_empty_row], inplace=True)

In [36]:
# Units of Measure (Metric or Imperial)
units = raw_values.loc[0, 'Units']
kms_per_mile = 1.609344

In [37]:
# Conversion formula 1.609344 kms per mile
if units == 'MPH':
    pass
else:
    course_df['Speed Req'] = round((course_df['Speed Req']/kms_per_mile), 0).astype(int)
    course_df['V1'] = round((course_df['V1']/kms_per_mile), 0).astype(int)    
    course_df['V2'] = round((course_df['V2']/kms_per_mile), 0).astype(int)    
    course_df['V3'] = round((course_df['V3']/kms_per_mile), 0).astype(int)    

In [38]:
# course_df.tail(80)
course_df.head(50)

,ID,Car #,Participant,Exercise,Speed Req,V1,V2,V3,Cones
0,1,4.0,Luis Alberto Zozaya Ordoñez,Slalom,28,30,27,26,NaN
1,2,2.0,German Enoc Velazco Gallardo,Slalom,28,30,29,32,NaN
2,3,3.0,Luis Alejandro Novelo Ramírez,Slalom,28,31,30,30,NaN
3,4,4.0,Héctor Ruiz Flores,Slalom,28,25,20,25,NaN
4,5,1.0,Jesús Cruz González,Slalom,28,27,24,27,NaN
5,6,4.0,Luis Alberto Zozaya Ordoñez,Slalom,31,34,29,34,NaN
6,7,2.0,German Enoc Velazco Gallardo,Slalom,31,35,32,29,NaN
7,8,3.0,Luis Alejandro Novelo Ramírez,Slalom,31,31,30,34,NaN
8,9,4.0,Héctor Ruiz Flores,Slalom,31,31,27,27,NaN
9,10,1.0,Jesús Cruz González,Slalom,31,31,28,30,NaN


In [39]:
course_df.tail(50)

,ID,Car #,Participant,Exercise,Speed Req,V1,V2,V3,Cones
83,84,4.0,Héctor Ruiz Flores,Lane Change,34,34,32,31,NaN
84,85,4.0,Luis Alberto Zozaya Ordoñez,Lane Change,34,37,35,34,1.0
85,86,1.0,German Enoc Velazco Gallardo,Lane Change,34,36,35,31,NaN
86,87,2.0,Jesús Cruz González,Lane Change,34,34,32,30,NaN
87,88,3.0,Luis Alejandro Novelo Ramírez,Lane Change,34,34,34,30,NaN
88,89,4.0,Héctor Ruiz Flores,Lane Change,34,34,33,29,NaN
89,90,4.0,Luis Alberto Zozaya Ordoñez,Lane Change,34,34,35,32,NaN
90,91,1.0,German Enoc Velazco Gallardo,Lane Change,34,34,33,32,NaN
91,92,2.0,Jesús Cruz González,Lane Change,34,35,34,31,NaN
92,93,3.0,Luis Alejandro Novelo Ramírez,Lane Change,34,35,34,32,NaN


In [40]:
# course_df.tail(50)

In [41]:
course_df.columns = course_df.columns.str.replace(' ', '_')
course_df.columns = (x.lower() for x in course_df.columns)
course_df.set_index('id', inplace=True)

In [42]:
course_df['cones'] = course_df['cones'].fillna(0)
course_df['cones'] = course_df['cones'].astype(int)
course_df['car_#'] = course_df['car_#'].astype(int)

In [43]:
exercise_values.columns = exercise_values.columns.str.replace(' ', '_')
exercise_values.columns = (x.lower() for x in exercise_values.columns)

In [44]:
class Students:

    num_of_students = 0

    def __init__(self, first, last, bdate):
        self.first = first
        self.last = last
        self.bdate = bdate
        self.bdate = (
            datetime.strptime(self.bdate, "%Y-%m-%d %H:%M:%S")
#             datetime.strptime(self.bdate, "%m/%d/%Y")

            .date()
            .strftime("%m-%d-%Y")
        )

        Students.num_of_students += 1

        self.unique_id = "{}{}{}{}".format(
            self.last[0].upper(),
            self.last[-1].upper(),
            self.first[0].upper(),
            self.birth_date(),
        )



    def birth_date(self):
        datetime_object = datetime.strptime(self.bdate, "%m-%d-%Y").date()
        date3 = datetime_object.strftime("%y%m%d")
        return date3

    def fullname(self):
        return "{} {}".format(self.first, self.last)

In [45]:
# Adding unique IDs to participants
unique_ids = []

def add_ids(data):
    for index, row in data.iterrows():
        drvr = Students(row.firstname, row.lastname, str(row.dob))
        unique_ids.append(drvr.unique_id)

In [46]:
students_df

,firstname,lastname,gender,dob
0,Juan,Guerrero Ortiz,M,1957-07-31
1,German Enoc,Velazco Gallardo,M,1977-03-07
2,Miguel Angel,Ortiz Herrera,M,1977-09-29
3,Jesús,Cruz González,M,1964-11-14
4,Luis Alberto,Zozaya Ordoñez,M,1993-01-05
...,...,...,...,...
213,NaN,NaN,NaN,NaT
214,NaN,NaN,NaN,NaT
215,NaN,NaN,NaN,NaT
216,NaN,NaN,NaN,NaT


In [47]:
"""
Clean empty rows covering for Excel BS
"""

first_emprty_row = students_df[students_df.isnull().all(axis=1) == True].index.tolist()
students_df.drop(students_df.index[first_emprty_row], inplace=True)
# students_df

In [48]:
add_ids(students_df)

In [49]:
students_df['Unique_Id'] = unique_ids

In [50]:
students_df

,firstname,lastname,gender,dob,Unique_Id
0,Juan,Guerrero Ortiz,M,1957-07-31,GZJ570731
1,German Enoc,Velazco Gallardo,M,1977-03-07,VOG770307
2,Miguel Angel,Ortiz Herrera,M,1977-09-29,OAM770929
3,Jesús,Cruz González,M,1964-11-14,CZJ641114
4,Luis Alberto,Zozaya Ordoñez,M,1993-01-05,ZZL930105
5,Fernando Alejandro,Rojas Barrios,M,1973-11-08,RSF731108
6,Luis Alejandro,Novelo Ramírez,M,1962-12-26,NZL621226
7,Luis Martín,Lima González,M,1963-02-10,LZL630210
8,Héctor,Ruiz Flores,M,1968-10-19,RSH681019


In [51]:
course_df

,car_#,participant,exercise,speed_req,v1,v2,v3,cones
id,,,,,,,,
1,4,Luis Alberto Zozaya Ordoñez,Slalom,28,30,27,26,0
2,2,German Enoc Velazco Gallardo,Slalom,28,30,29,32,0
3,3,Luis Alejandro Novelo Ramírez,Slalom,28,31,30,30,0
4,4,Héctor Ruiz Flores,Slalom,28,25,20,25,0
5,1,Jesús Cruz González,Slalom,28,27,24,27,0
...,...,...,...,...,...,...,...,...
129,2,Fernando Alejandro Rojas Barrios,Lane Change,34,34,34,34,0
130,3,Luis Martín Lima González,Lane Change,34,34,34,29,0
131,4,Juan Guerrero Ortiz,Lane Change,34,39,37,32,0


In [52]:
students_df.set_index('Unique_Id', inplace=True)

In [53]:
students_df.firstname = students_df.firstname.str.strip()
students_df.lastname = students_df.lastname.str.strip()

In [54]:
students_df['fullname'] = students_df.apply(lambda row: row['firstname'] + ' ' + row['lastname'], axis=1)

In [55]:
percetages_of_ex = []

def ex_percetage(data):
    for index, row in data.iterrows():
        x = row[['v1', 'v2', 'v3']].mean()
        sd = row[['v1', 'v2', 'v3']].std()
        cns = row['cones']
        y = round(x, 2)
        z = row['speed_req']
        p = ((y / z))
        res = round(p, 2)
        if sd < 3 and cns == 0:
            pass
        else:
            res = 0
        percetages_of_ex.append(res)

In [56]:
# Calculations call
ex_percetage(course_df)

In [57]:
# percetages_of_ex

In [58]:
course_df['%_of_exercise'] = percetages_of_ex

In [59]:
Car_1 = course_cars.loc[1, 'LatAcc']

In [60]:
"""We are seeing Gs not percetages

Calculates LA, filters any result that shows above 100% and defaults to 0.000
Claculates a SD of 3 and disqualifies anyone not within that
"""

percetages_of_v = []

def v_percetage(data):
    for index, row in data.iterrows():
        vx = row[['v1', 'v2', 'v3']].mean()
        sd = row[['v1', 'v2', 'v3']].std()
        ex = row['exercise']
        R = exercise_values.loc[ex, 'radius']
        v = round(vx, 2)
        LA = ((v**2) / (R*15))
        res = round(LA, 2) / course_cars.loc[1]['LatAcc']
        if res < 1:
            pass
        else:
            res = 0
        if sd < 3:
            pass
        else:
            res = 0
        percetages_of_v.append(res)

In [61]:
# Calculations call
v_percetage(course_df)

In [62]:
course_df['%_of_vehicle'] = percetages_of_v

In [63]:
counts_df = pd.DataFrame(course_df[['participant', 'exercise']])
counts_df['%_of_exercise'] = course_df['%_of_exercise'].astype('float')
counts_df['%_of_vehicle'] = course_df['%_of_vehicle'].astype('float')

In [64]:
# course_df.loc[course_df['%_of_vehicle'] >= .8]
course_df.head(50)

,car_#,participant,exercise,speed_req,v1,v2,v3,cones,%_of_exercise,%_of_vehicle
id,,,,,,,,,,
1,4,Luis Alberto Zozaya Ordoñez,Slalom,28,30,27,26,0,0.99,0.623656
2,2,German Enoc Velazco Gallardo,Slalom,28,30,29,32,0,1.08,0.752688
3,3,Luis Alejandro Novelo Ramírez,Slalom,28,31,30,30,0,1.08,0.752688
4,4,Héctor Ruiz Flores,Slalom,28,25,20,25,0,0.83,0.440860
5,1,Jesús Cruz González,Slalom,28,27,24,27,0,0.93,0.548387
6,4,Luis Alberto Zozaya Ordoñez,Slalom,31,34,29,34,0,1.04,0.849462
7,2,German Enoc Velazco Gallardo,Slalom,31,35,32,29,0,0.00,0.000000
8,3,Luis Alejandro Novelo Ramírez,Slalom,31,31,30,34,0,1.02,0.817204
9,4,Héctor Ruiz Flores,Slalom,31,31,27,27,0,0.91,0.655914


### New Calculation for Max Score <--- Sept 2021

In [65]:
passed_ex_df = pd.DataFrame(course_df[['participant','exercise','cones','%_of_vehicle']]).reset_index(drop=True)
failed_ex = passed_ex_df.loc[(passed_ex_df['cones'] >= 1) | (passed_ex_df['%_of_vehicle'] < .1)].index.tolist()
passed_ex_df.drop(passed_ex_df.index[failed_ex], inplace=True)
passed_ex_df.reset_index(drop=True)
passed_ex_df = pd.DataFrame(passed_ex_df
                            .groupby(['participant', 'exercise'])
                           .agg({'%_of_vehicle':['min', 'max']})
                           .rename(columns={'min':'Start Score', 'max':'End Score'})
                           .unstack(level=1))
passed_ex_df.columns = passed_ex_df.columns.droplevel(0)
passed_ex_df

Start Score             End Score          
exercise                         Lane Change    Slalom Lane Change    Slalom
participant                                                                 
Fernando Alejandro Rojas Barrios    0.688172  0.698925    0.838710  0.881720
German Enoc Velazco Gallardo        0.763441  0.720430    0.838710  0.881720
Héctor Ruiz Flores                  0.741935  0.440860    0.892473  0.795699
Jesús Cruz González                 0.741935  0.548387    0.806452  0.838710
Juan Guerrero Ortiz                 0.806452  0.698925    0.860215  0.784946
Luis Alberto Zozaya Ordoñez         0.741935  0.623656    0.946237  0.870968
Luis Alejandro Novelo Ramírez       0.774194  0.752688    0.892473  0.870968
Luis Martín Lima González           0.763441  0.784946    0.860215  0.838710
Miguel Angel Ortiz Herrera          0.774194  0.666667    0.924731  0.903226

In [66]:
counts_df = pd.DataFrame(counts_df.replace(0, np.nan)
                    .groupby(['participant', 'exercise'])
                    .agg({'exercise':'size','%_of_exercise':['count', 'mean'],'%_of_vehicle':['min', 'max']})
                    .rename(columns={'size':'Count','count':'Passed','mean':'Av Score','min':'Start Score', 'max':'End Score'})).unstack(level=1)
counts_df.columns = counts_df.columns.droplevel(0)


In [67]:
counts_df = counts_df.replace(np.nan, 0)

In [68]:
counts_df

Count             Passed         \
exercise                         Lane Change Slalom Lane Change Slalom   
participant                                                              
Fernando Alejandro Rojas Barrios           7      7           6      6   
German Enoc Velazco Gallardo              10      8           7      6   
Héctor Ruiz Flores                         7      8           4      7   
Jesús Cruz González                        7      8           5      7   
Juan Guerrero Ortiz                        6      9           3      6   
Luis Alberto Zozaya Ordoñez                7      8           5      8   
Luis Alejandro Novelo Ramírez              7      8           5      7   
Luis Martín Lima González                  7      9           7      5   
Miguel Angel Ortiz Herrera                 5      5           5      2   

                                    Av Score           Start Score            \
exercise                         Lane Change    Slalom Lane Change    Slalom   
participant                                                                    
Fernando Alejandro Rojas Barrios    0.988333  1.020000    0.688172  0.698925   
German Enoc Velazco Gallardo        0.982857  1.023333    0.763441  0.720430   
Héctor Ruiz Flores                  0.977500  0.957143    0.741935  0.440860   
Jesús Cruz González                 0.966000  0.975714    0.741935  0.548387   
Juan Guerrero Ortiz                 1.030000  0.973333    0.806452  0.698925   
Luis Alberto Zozaya Ordoñez         1.032000  1.011250    0.741935  0.623656   
Luis Alejandro Novelo Ramírez       0.994000  1.022857    0.774194  0.752688   
Luis Martín Lima González           0.988571  1.014000    0.763441  0.655914   
Miguel Angel Ortiz Herrera          1.034000  1.050000    0.774194  0.666667   

                                   End Score            
exercise                         Lane Change    Slalom  
participant                                             
Fernando Alejandro Rojas Barrios    0.838710  0.881720  
German Enoc Velazco Gallardo        0.838710  0.881720  
Héctor Ruiz Flores                  0.892473  0.795699  
Jesús Cruz González                 0.806452  0.838710  
Juan Guerrero Ortiz                 0.892473  0.784946  
Luis Alberto Zozaya Ordoñez         0.946237  0.870968  
Luis Alejandro Novelo Ramírez       0.892473  0.870968  
Luis Martín Lima González           0.860215  0.838710  
Miguel Angel Ortiz Herrera          0.924731  0.903226

In [69]:
# Slalom average
slalom_avg = pd.DataFrame(course_df.replace(0, np.nan)
                    .groupby(['participant', 'exercise'])
                    .agg({'%_of_vehicle':'mean'})
                    .rename(columns={'%_of_vehicle':'vehicle_pc_avg'})).unstack(level=1)
# counts_df.columns = counts_df.columns.droplevel(0)

In [70]:
finalx_df

,ID,Exercise,Participant,Pressure,Car #,Cones,Doors,Time,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,1,Final Exercise,Demo,0.0,1.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Final Exercise,Demo,0.0,1.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Final Exercise,Demo,0.0,1.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Final Exercise,Miguel Angel Ortiz Herrera,0.0,1.0,1.0,NaN,70.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Final Exercise,Luis Alberto Zozaya Ordoñez,0.0,1.0,NaN,NaN,76.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Final Exercise,Héctor Ruiz Flores,0.0,1.0,1.0,NaN,82.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Final Exercise,Jesús Cruz González,0.0,1.0,4.0,NaN,81.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Final Exercise,Juan Guerrero Ortiz,0.0,1.0,4.0,NaN,73.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Final Exercise,German Enoc Velazco Gallardo,0.0,1.0,3.0,NaN,79.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Final Exercise,Luis Martín Lima González,0.0,1.0,3.0,1.0,89.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
LnCh_count = counts_df['Count']['Lane Change']
LnCh_passed = counts_df['Passed']['Lane Change']
counts_df['LnCh Passed'] = (LnCh_passed / LnCh_count)
slalom_count = counts_df['Count']['Slalom']
slalom_passed = counts_df['Passed']['Slalom']
counts_df['Slalom Passed'] = (slalom_passed / slalom_count)

In [72]:
counts_df.loc[counts_df['End Score']['Slalom'] < .8] #People that did not get to 80%

Count             Passed           Av Score  \
exercise            Lane Change Slalom Lane Change Slalom Lane Change   
participant                                                             
Héctor Ruiz Flores            7      8           4      7      0.9775   
Juan Guerrero Ortiz           6      9           3      6      1.0300   

                              Start Score             End Score            \
exercise               Slalom Lane Change    Slalom Lane Change    Slalom   
participant                                                                 
Héctor Ruiz Flores   0.957143    0.741935  0.440860    0.892473  0.795699   
Juan Guerrero Ortiz  0.973333    0.806452  0.698925    0.892473  0.784946   

                    LnCh Passed Slalom Passed  
exercise                                       
participant                                    
Héctor Ruiz Flores     0.571429      0.875000  
Juan Guerrero Ortiz    0.500000      0.666667

In [73]:
finalx_df.rename(columns={'Exercise':'exercise', 'Participant':'participant', 'Car' : 'car_ID', 'Pressure':'stress', 'Reverse Slalom':'rev_slalom', 'SV1':'s_v1',
       'SV2':'s_v2', 'SV3':'s_v3', 'LChV1':'LCh_v1', 'LChV2':'LCh_v2', 'LChV3':'LCh_v3', 'Cones':'cones', 'Doors':'gates', 'Time':'g_time'}, inplace=True)

In [75]:
finalx_df

,ID,exercise,participant,stress,Car #,cones,gates,g_time,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,1,Final Exercise,Demo,0.0,1.0,NaN,NaN,0 days 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Final Exercise,Demo,0.0,1.0,NaN,NaN,0 days 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Final Exercise,Demo,0.0,1.0,NaN,NaN,0 days 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Final Exercise,Miguel Angel Ortiz Herrera,0.0,1.0,1.0,NaN,0 days 00:01:10.330000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Final Exercise,Luis Alberto Zozaya Ordoñez,0.0,1.0,NaN,NaN,0 days 00:01:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Final Exercise,Héctor Ruiz Flores,0.0,1.0,1.0,NaN,0 days 00:01:22.030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Final Exercise,Jesús Cruz González,0.0,1.0,4.0,NaN,0 days 00:01:21.420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Final Exercise,Juan Guerrero Ortiz,0.0,1.0,4.0,NaN,0 days 00:01:13.390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Final Exercise,German Enoc Velazco Gallardo,0.0,1.0,3.0,NaN,0 days 00:01:19.170000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Final Exercise,Luis Martín Lima González,0.0,1.0,3.0,1.0,0 days 00:01:29.140000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
#Time conversion to Time Delta
finalx_df['g_time'] = pd.to_timedelta(finalx_df['g_time'], unit='s')
# finalx_df['rev_slalom'] = pd.to_timedelta(finalx_df['rev_slalom'], unit='s')
finalx_df[['cones', 'gates']] = finalx_df[['cones', 'gates']].fillna(0).astype(int)

In [77]:
finalx_df.drop('ID', axis=1, inplace=True)

In [78]:
finalx_df
# counts_df

,exercise,participant,stress,Car #,cones,gates,g_time,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,Final Exercise,Demo,0.0,1.0,0,0,0 days 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Final Exercise,Demo,0.0,1.0,0,0,0 days 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Final Exercise,Demo,0.0,1.0,0,0,0 days 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Final Exercise,Miguel Angel Ortiz Herrera,0.0,1.0,1,0,0 days 00:01:10.330000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Final Exercise,Luis Alberto Zozaya Ordoñez,0.0,1.0,0,0,0 days 00:01:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Final Exercise,Héctor Ruiz Flores,0.0,1.0,1,0,0 days 00:01:22.030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Final Exercise,Jesús Cruz González,0.0,1.0,4,0,0 days 00:01:21.420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Final Exercise,Juan Guerrero Ortiz,0.0,1.0,4,0,0 days 00:01:13.390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Final Exercise,German Enoc Velazco Gallardo,0.0,1.0,3,0,0 days 00:01:19.170000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Final Exercise,Luis Martín Lima González,0.0,1.0,3,1,0 days 00:01:29.140000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
# Final exercise variables

ideal_time = raw_values.loc[0, 'Ideal_Time Sec']
c_penalty = raw_values.loc[0, 'Cone Penalty Sec']
g_penalty = raw_values.loc[0, 'Door Penalty Sec']
ideal_time = pd.to_timedelta(ideal_time, unit='s')
c_penalty = pd.to_timedelta(c_penalty, unit='s')
g_penalty = pd.to_timedelta(g_penalty, unit='s')

In [80]:
# Final exercise - final time calculations

final_fx = []

def fx_fnl_res(data):
    for index, row in data.iterrows():
#         sp = row[['s_v1', 's_v2', 's_v3']].mean()
#         lcp = row[['LCh_v1', 'LCh_v2', 'LCh_v3']].mean()
        cns = row['cones']
        gts = row['gates']
        tme1 = row['g_time']
        g_tme = (tme1 + (cns * c_penalty) + (gts * g_penalty))
        f_time = (-(((g_tme / ideal_time)*100)-200)/100)
        f_time = round(f_time, 2)
        final_fx.append(f_time)

In [81]:
fx_fnl_res(finalx_df)

In [82]:
ideal_time
it1 = ideal_time.total_seconds()

In [83]:
finalx_df['f_time'] = finalx_df.apply(lambda row: row['g_time'] + ((row['cones']*c_penalty) + (row['gates']*g_penalty)), axis=1)
# finalx_df['final_result'] = finalx_df.apply(lambda row: ((((row['f_time'].total_seconds())/it1)*100)-200), axis=1)

In [84]:
# VBox Data procesing

# Car 1 DataFrame
c1_df.rename(columns={'UTC time (At Start)':'UTC_Time'}, inplace=True)
# c1_df.drop(c1_df.index[-4:], inplace=True)
c1_df.rename(columns={'Time (s)' : 'rev_slalom'}, inplace=True)
c1_rs_time = c1_df[['car', 'rev_slalom', 'UTC_Time']][::3]
c1_rs_time.reset_index(drop=True, inplace=True)

# Slalom Structure
c1_slalom = c1_df[['Speed (Avg) (mph)', 'Speed (Std Dev) (mph)']][1::3]
c1_slalom.reset_index(drop=True, inplace=True)
c1_slalom.rename(columns={
    'Speed (Avg) (mph)': 'slalom_avg', 
    'Speed (Std Dev) (mph)':'slalom_std_dev', 
#     'UTC time (At Start)':'UTC_Time'
}, inplace=True)

# Lane Change Structure
c1_LnCh = c1_df[['Speed (Avg) (mph)', 'Speed (Std Dev) (mph)']][2::3]
c1_LnCh.rename(columns={
    'Speed (Avg) (mph)': 'LnCh_avg', 
    'Speed (Std Dev) (mph)':'LnCh_std_dev',
#     'UTC time (At Start)':'UTC_Time'
}, inplace=True)
c1_LnCh.reset_index(drop=True, inplace=True)

# Frame variables
c1_frames = [c1_rs_time, c1_slalom, c1_LnCh]

# Final DataFrames
c1_result = pd.concat(c1_frames, join='inner', axis=1)

# Creating Time Deltas
c1_result['rev_slalom'] = c1_result['rev_slalom'].astype(float)
c1_result['rev_slalom'] = pd.to_timedelta(c1_result['rev_slalom'], unit='s')

# Converting time to datetime64
c1_result['UTC_Time'] = (pd.to_datetime(c1_result['UTC_Time'], format=('%H:%M:%S.%f')).dt.time)

# Changind types to float
c1_result[['slalom_avg', 'slalom_std_dev', 'LnCh_avg', 'LnCh_std_dev']] = c1_result[['slalom_avg', 'slalom_std_dev', 'LnCh_avg', 'LnCh_std_dev']].astype(float)
c1_result.rename(columns={'car' : 'car_ID'}, inplace=True)
c1_result['car_ID'] = c1_result['car_ID'].astype(int)


# Car 2 DataFrame
c2_df.rename(columns={'UTC time (At Start)':'UTC_Time'}, inplace=True)
c2_df.drop(c2_df.index[-4:], inplace=True)
c2_df.rename(columns={'Time (s)' : 'rev_slalom'}, inplace=True)
c2_rs_time = c2_df[['car','rev_slalom', 'UTC_Time']][::3]
c2_rs_time.reset_index(drop=True, inplace=True)

# Slalom Structure
c2_slalom = c2_df[['Speed (Avg) (mph)', 'Speed (Std Dev) (mph)']][1::3]
c2_slalom.reset_index(drop=True, inplace=True)
c2_slalom.rename(columns={
    'Speed (Avg) (mph)': 'slalom_avg', 
    'Speed (Std Dev) (mph)':'slalom_std_dev', 
#     'UTC time (At End)':'UTC_Time'
}, inplace=True)

# Lane Change Structure
c2_LnCh = c2_df[['Speed (Avg) (mph)', 'Speed (Std Dev) (mph)']][2::3]
c2_LnCh.rename(columns={
    'Speed (Avg) (mph)': 'LnCh_avg', 
    'Speed (Std Dev) (mph)':'LnCh_std_dev',
#     'UTC time (At End)':'UTC_Time'
}, inplace=True)
c2_LnCh.reset_index(drop=True, inplace=True)

# Frame variables
c2_frames = [c2_rs_time, c2_slalom, c2_LnCh]

# Final DataFrames
c2_result = pd.concat(c2_frames, join='inner', axis=1)

# Creating Time Deltas
c2_result['rev_slalom'] = pd.to_timedelta(c2_result['rev_slalom'], unit='s')

# Converting time to datetime64
c2_result['UTC_Time'] = (pd.to_datetime(c2_result['UTC_Time'], format=('%H:%M:%S.%f')).dt.time)

# Changind types to float
c2_result[['slalom_avg', 'slalom_std_dev', 'LnCh_avg', 'LnCh_std_dev']] = c2_result[['slalom_avg', 'slalom_std_dev', 'LnCh_avg', 'LnCh_std_dev']].astype(float)
c2_result.rename(columns={'car' : 'car_ID'}, inplace=True)
c2_result['car_ID'] = c2_result['car_ID'].astype(int)


In [85]:
# Car 3 DataFrame
c3_df.rename(columns={'UTC time (At Start)':'UTC_Time'}, inplace=True)
# c3_df.drop(c3_df.index[-4:], inplace=True)
c3_df.rename(columns={'Time (s)' : 'rev_slalom'}, inplace=True)
c3_rs_time = c3_df[['car','rev_slalom', 'UTC_Time']][::3]
c3_rs_time.reset_index(drop=True, inplace=True)

# Slalom Structure
c3_slalom = c3_df[['Speed (Avg) (mph)', 'Speed (Std Dev) (mph)']][1::3]
c3_slalom.reset_index(drop=True, inplace=True)
c3_slalom.rename(columns={
    'Speed (Avg) (mph)': 'slalom_avg', 
    'Speed (Std Dev) (mph)':'slalom_std_dev', 
#     'UTC time (At End)':'UTC_Time'
}, inplace=True)

# Lane Change Structure
c3_LnCh = c3_df[['Speed (Avg) (mph)', 'Speed (Std Dev) (mph)']][2::3]
c3_LnCh.rename(columns={
    'Speed (Avg) (mph)': 'LnCh_avg', 
    'Speed (Std Dev) (mph)':'LnCh_std_dev',
#     'UTC time (At End)':'UTC_Time'
}, inplace=True)
c3_LnCh.reset_index(drop=True, inplace=True)

# Frame variables
c3_frames = [c3_rs_time, c3_slalom, c3_LnCh]

# Final DataFrames
c3_result = pd.concat(c3_frames, join='inner', axis=1)

# Creating Time Deltas
c3_result['rev_slalom'] = c3_result['rev_slalom'].astype(float)
c3_result['rev_slalom'] = pd.to_timedelta(c3_result['rev_slalom'], unit='s')

# Converting time to datetime64
c3_result['UTC_Time'] = (pd.to_datetime(c3_result['UTC_Time'], format=('%H:%M:%S.%f')).dt.time)

# Changind types to float
c3_result[['slalom_avg', 'slalom_std_dev', 'LnCh_avg', 'LnCh_std_dev']] = c3_result[['slalom_avg', 'slalom_std_dev', 'LnCh_avg', 'LnCh_std_dev']].astype(float)
c3_result.rename(columns={'car' : 'car_ID'}, inplace=True)
c3_result['car_ID'] = c3_result['car_ID'].astype(int)

In [86]:
# Creating a sigle DataFrame
vbox_df = pd.concat([c1_result, c2_result, c3_result], ignore_index=True).sort_values(by=['UTC_Time']).reset_index().drop(['index'], axis=1)

In [87]:
# vbox_df

In [88]:
finalx_df['rev_slalom'] = vbox_df['rev_slalom']

In [89]:
# vbox_df.head()
# c1_result

In [90]:
# MSE slalom and LnCh percentages
mse_slalom_prcnt = []
mse_LnCh_prcnt = []

def mse_slalom_pc(data):
    for index, row in data.iterrows():
        vx = row['slalom_avg']
        ex = 'Slalom'
        R = exercise_values.loc[ex, 'radius']
        v = round(vx, 2)
        LA = ((v**2) / (R*15))
        if LA >= course_cars.loc[1]['LatAcc']:
            res = 0
        else:
            res = round(LA, 2) / course_cars.loc[1]['LatAcc']
        mse_slalom_prcnt.append(int((round(res, 2))*100))

def mse_LnCh_pc(data):
    for index, row in data.iterrows():
        vx = row['LnCh_avg']
        ex = 'Lane Change'
        R = exercise_values.loc[ex, 'radius']
        v = round(vx, 2)
        LA = ((v**2) / (R*15))
        if LA >= course_cars.loc[1]['LatAcc']:
            res = 0
        else:
            res = round(LA, 2) / course_cars.loc[1]['LatAcc']
        mse_LnCh_prcnt.append(int((round(res, 2))*100))

In [91]:
# vbox_df

In [92]:
# vbox_df
# mse_slalom_prcnt
# finalx_df

In [93]:
mse_slalom_pc(vbox_df)
mse_LnCh_pc(vbox_df)
finalx_df['slalom'] = mse_slalom_prcnt
finalx_df['LnCh'] = mse_LnCh_prcnt

In [94]:
finalx_df['final_result'] = final_fx

In [95]:
demo_rows = finalx_df[finalx_df['participant'] == 'Demo'].index
finalx_df.drop(demo_rows, inplace=True)
# finalx_df

In [96]:
finalx_df['rev_%'] = finalx_df.apply(lambda x: round((x['rev_slalom'] / x['g_time']), 2), axis=1)

In [100]:
finalx_df = finalx_df[['exercise', 'participant', 'Car #', 'stress', 'rev_slalom', 'rev_%', 'slalom', 'LnCh', 'cones', 'gates',
       'f_time', 'final_result']]

In [101]:
finalx_df.reset_index(drop=True, inplace=True)

In [102]:
finalx_df

,exercise,participant,Car #,stress,rev_slalom,rev_%,slalom,LnCh,cones,gates,f_time,final_result
0,Final Exercise,Miguel Angel Ortiz Herrera,1.0,0.0,0 days 00:00:14.300000,0.20,71,74,1,0,0 days 00:01:12.330000,0.95
1,Final Exercise,Luis Alberto Zozaya Ordoñez,1.0,0.0,0 days 00:00:15.810000,0.21,61,54,0,0,0 days 00:01:16,0.90
2,Final Exercise,Héctor Ruiz Flores,1.0,0.0,0 days 00:00:19.450000,0.24,52,43,1,0,0 days 00:01:24.030000,0.78
3,Final Exercise,Jesús Cruz González,1.0,0.0,0 days 00:00:18.090000,0.22,70,60,4,0,0 days 00:01:29.420000,0.70
4,Final Exercise,Juan Guerrero Ortiz,1.0,0.0,0 days 00:00:15.730000,0.21,72,54,4,0,0 days 00:01:21.390000,0.82
5,Final Exercise,German Enoc Velazco Gallardo,1.0,0.0,0 days 00:00:17.950000,0.23,74,82,3,0,0 days 00:01:25.170000,0.77
6,Final Exercise,Luis Martín Lima González,1.0,0.0,0 days 00:00:22.190000,0.25,48,59,3,1,0 days 00:01:40.140000,0.55
7,Final Exercise,Fernando Alejandro Rojas Barrios,1.0,0.0,0 days 00:00:17.570000,0.23,73,46,0,0,0 days 00:01:15.670000,0.90
8,Final Exercise,Luis Alejandro Novelo Ramírez,1.0,0.0,0 days 00:00:15.900000,0.21,65,57,1,0,0 days 00:01:17.340000,0.88
9,Final Exercise,Luis Martín Lima González,1.0,1.0,0 days 00:00:17.400000,0.22,62,66,3,1,0 days 00:01:28.810000,0.71


## Report

In [108]:
course_df

,car_#,participant,exercise,speed_req,v1,v2,v3,cones,%_of_exercise,%_of_vehicle
id,,,,,,,,,,
1,4,Luis Alberto Zozaya Ordoñez,Slalom,28,30,27,26,0,0.99,0.623656
2,2,German Enoc Velazco Gallardo,Slalom,28,30,29,32,0,1.08,0.752688
3,3,Luis Alejandro Novelo Ramírez,Slalom,28,31,30,30,0,1.08,0.752688
4,4,Héctor Ruiz Flores,Slalom,28,25,20,25,0,0.83,0.440860
5,1,Jesús Cruz González,Slalom,28,27,24,27,0,0.93,0.548387
...,...,...,...,...,...,...,...,...,...,...
129,2,Fernando Alejandro Rojas Barrios,Lane Change,34,34,34,34,0,1.00,0.838710
130,3,Luis Martín Lima González,Lane Change,34,34,34,29,0,0.95,0.763441
131,4,Juan Guerrero Ortiz,Lane Change,34,39,37,32,0,0.00,0.000000


In [110]:
counts_df

Count             Passed         \
exercise                         Lane Change Slalom Lane Change Slalom   
participant                                                              
Fernando Alejandro Rojas Barrios           7      7           6      6   
German Enoc Velazco Gallardo              10      8           7      6   
Héctor Ruiz Flores                         7      8           4      7   
Jesús Cruz González                        7      8           5      7   
Juan Guerrero Ortiz                        6      9           3      6   
Luis Alberto Zozaya Ordoñez                7      8           5      8   
Luis Alejandro Novelo Ramírez              7      8           5      7   
Luis Martín Lima González                  7      9           7      5   
Miguel Angel Ortiz Herrera                 5      5           5      2   

                                    Av Score           Start Score            \
exercise                         Lane Change    Slalom Lane Change    Slalom   
participant                                                                    
Fernando Alejandro Rojas Barrios    0.988333  1.020000    0.688172  0.698925   
German Enoc Velazco Gallardo        0.982857  1.023333    0.763441  0.720430   
Héctor Ruiz Flores                  0.977500  0.957143    0.741935  0.440860   
Jesús Cruz González                 0.966000  0.975714    0.741935  0.548387   
Juan Guerrero Ortiz                 1.030000  0.973333    0.806452  0.698925   
Luis Alberto Zozaya Ordoñez         1.032000  1.011250    0.741935  0.623656   
Luis Alejandro Novelo Ramírez       0.994000  1.022857    0.774194  0.752688   
Luis Martín Lima González           0.988571  1.014000    0.763441  0.655914   
Miguel Angel Ortiz Herrera          1.034000  1.050000    0.774194  0.666667   

                                   End Score           LnCh Passed  \
exercise                         Lane Change    Slalom               
participant                                                          
Fernando Alejandro Rojas Barrios    0.838710  0.881720    0.857143   
German Enoc Velazco Gallardo        0.838710  0.881720    0.700000   
Héctor Ruiz Flores                  0.892473  0.795699    0.571429   
Jesús Cruz González                 0.806452  0.838710    0.714286   
Juan Guerrero Ortiz                 0.892473  0.784946    0.500000   
Luis Alberto Zozaya Ordoñez         0.946237  0.870968    0.714286   
Luis Alejandro Novelo Ramírez       0.892473  0.870968    0.714286   
Luis Martín Lima González           0.860215  0.838710    1.000000   
Miguel Angel Ortiz Herrera          0.924731  0.903226    1.000000   

                                 Slalom Passed  
exercise                                        
participant                                     
Fernando Alejandro Rojas Barrios      0.857143  
German Enoc Velazco Gallardo          0.750000  
Héctor Ruiz Flores                    0.875000  
Jesús Cruz González                   0.875000  
Juan Guerrero Ortiz                   0.666667  
Luis Alberto Zozaya Ordoñez           1.000000  
Luis Alejandro Novelo Ramírez         0.875000  
Luis Martín Lima González             0.555556  
Miguel Angel Ortiz Herrera            0.400000

In [103]:
# Group Variables
gasnor = int((counts_df['Count']['Slalom'].agg('mean'))) # group_average_slalom_runs
gaspoce = int((counts_df['Slalom Passed'].mean())*100) # group_average_slalom_prcnt_completed
gasaoep = int(((counts_df['Av Score']['Slalom'].mean())*100)) # group_average_slalom_ex_prcnt
gasaovc = int((course_df.loc[course_df['exercise'] == 'Slalom']['%_of_vehicle'].mean())*100) # group_average_slalom_vehicle_control
galnor = int((counts_df['Count']['Lane Change'].agg('mean'))) #.astype(int) # group_average_lnch_runs
galpoce = int((counts_df['LnCh Passed'].mean())*100) # group_average_lnch_prcnt_completed
galaoep = int(((counts_df['Av Score']['Lane Change'].mean())*100)) # group_average_lnch_ex_prcnt
galaovc = int((course_df.loc[course_df['exercise'] == 'Lane Change']['%_of_vehicle'].mean())*100) # group_average_lnch_vehicle_control
mseg_t_pre = str(finalx_df['f_time'].mean())
mseg_t = mseg_t_pre[mseg_t_pre.find(':')+1 : mseg_t_pre.find('.')+3 : ] # mse_group_av_time
mseg_c = int((finalx_df['cones'].mean())) # mse_group_av_cones
mseg_g = int((finalx_df['gates'].mean())) # mse_group_av_gates
mseg_perf = int((finalx_df['final_result'].mean())*100) # mse_group_av_performance
mseg_per = int((finalx_df['final_result'].quantile())*100) # mse_group_av_percentile
mse_obj = str(raw_values['Ideal_Time Sec'][0])[10 : : ] # mse_objective_obj
mseg_rev_pre = str(finalx_df['rev_slalom'].mean()) # strings mse_group_used_in_rev
"""New Change in slicing"""
mseg_rev = re.findall('days 00:00:(.*\..{2}|.*$)', mseg_rev_pre)[0] # strings mse_group_used_in_rev <- Chnaged May 2021
mseg_rev_pc = int(round(((finalx_df['rev_%'].mean())*100), 0))

In [96]:
finalx_df['rev_slalom'].mean()

Timedelta('0 days 00:00:16.379166666')

In [97]:
# Temporary Variables waiting for code
msed_per = np.nan # MSE Driver Percentile (Quantile)
msed_rev = np.nan # MSE Driver % Used in Reverse
msed_rev_time = np.nan # MSE Driver Reverse Time
paragraph = np.nan # Lead Instructor Feedback

In [98]:
# students_df
# raw_values
# counts_df
# course_df.head(50)
finalx_df
# slalom_avg
# slalom_avg
# report_df

,exercise,participant,car_ID,stress,rev_slalom,rev_%,slalom,LnCh,cones,gates,f_time,final_result
0,Final Exercise,Miguel Angel Ortiz Herrera,1.0,0.0,0 days 00:00:14.300000,0.20,71,74,1,0,0 days 00:01:12.330000,0.95
1,Final Exercise,Luis Alberto Zozaya Ordoñez,1.0,0.0,0 days 00:00:15.810000,0.21,61,54,0,0,0 days 00:01:16,0.90
2,Final Exercise,Héctor Ruiz Flores,1.0,0.0,0 days 00:00:19.450000,0.24,52,43,1,0,0 days 00:01:24.030000,0.78
3,Final Exercise,Jesús Cruz González,1.0,0.0,0 days 00:00:18.090000,0.22,70,60,4,0,0 days 00:01:29.420000,0.70
4,Final Exercise,Juan Guerrero Ortiz,1.0,0.0,0 days 00:00:15.730000,0.21,72,54,4,0,0 days 00:01:21.390000,0.82
5,Final Exercise,Enoc Velazco Gallardo,1.0,0.0,0 days 00:00:17.950000,0.23,74,82,3,0,0 days 00:01:25.170000,0.77
6,Final Exercise,Luis Martín Lima González,1.0,0.0,0 days 00:00:22.190000,0.25,48,59,3,1,0 days 00:01:40.140000,0.55
7,Final Exercise,Fernando Alejandro Rojas Barrios,1.0,0.0,0 days 00:00:17.570000,0.23,73,46,0,0,0 days 00:01:15.670000,0.90
8,Final Exercise,Luis Alejandro Novelo Ramírez,1.0,0.0,0 days 00:00:15.900000,0.21,65,57,1,0,0 days 00:01:17.340000,0.88
9,Final Exercise,Luis Martín Lima González,1.0,1.0,0 days 00:00:17.400000,0.22,62,66,3,1,0 days 00:01:28.810000,0.71


In [99]:
#Final report DataFrame Lineup
report_df = pd.DataFrame(students_df, columns=['fullname'])
report_df['company'] = np.nan
report_df['program'] = raw_values.loc[0, 'Program']
report_df['date'] = raw_values.loc[0, 'Date']

# Versiones en ambos idiomas
if raw_values.loc[0, 'Country'] == 'MX':
    report_df['vehicle'] = (raw_values.loc[0, 'Make'] + ' (Capacidad ' + raw_values.loc[0, 'LatAcc'].astype(str) + 'g)')
else:
    report_df['vehicle'] = (raw_values.loc[0, 'Make'] + ' (' + raw_values.loc[0, 'LatAcc'].astype(str) + 'g Capability)')

# Slalom variables
report_df['s_no_runs'] = pd.merge(left=report_df, right=(counts_df['Count']['Slalom']).astype(int), left_on='fullname', right_index=True)['Slalom']
report_df['s_passed'] = pd.merge(left=report_df, right=counts_df['Passed']['Slalom'], left_on='fullname', right_index=True)['Slalom']
report_df['prcnt_s_pass'] = pd.merge(left=report_df, right=round((counts_df['Slalom Passed'])*100, 0), left_on='fullname', right_index=True)['Slalom Passed']
report_df['avg_ex_control_s'] = pd.merge(left=report_df, right=round((counts_df['Av Score']['Slalom'])*100, 0), left_on='fullname', right_index=True)['Slalom']
report_df['avg_v_control_s'] = pd.merge(left=report_df, right=round((slalom_avg['vehicle_pc_avg']['Slalom'])*100, 0), left_on='fullname', right_index=True)['Slalom']
report_df['slalom_max'] = pd.merge(left=report_df, right=round((passed_ex_df['End Score']['Slalom'])*100), left_on='fullname', right_index=True)['Slalom']

#LnCh Variables
report_df['lc_no_runs'] = pd.merge(left=report_df, right=(counts_df['Count']['Lane Change']).astype(int), left_on='fullname', right_index=True)['Lane Change']
report_df['lc_passed'] = pd.merge(left=report_df, right=counts_df['Passed']['Lane Change'], left_on='fullname', right_index=True)['Lane Change']
report_df['prcnt_lc_pass'] = pd.merge(left=report_df, right=round((counts_df['LnCh Passed'])*100, 0), left_on='fullname', right_index=True)['LnCh Passed']
report_df['avg_ex_control_lc'] = pd.merge(left=report_df, right=round((counts_df['Av Score']['Lane Change'])*100, 0), left_on='fullname', right_index=True)['Lane Change']
report_df['avg_v_control_lc'] = pd.merge(left=report_df, right=round((slalom_avg['vehicle_pc_avg']['Lane Change'])*100, 0), left_on='fullname', right_index=True)['Lane Change']
report_df['lnch_max'] = pd.merge(left=report_df, right=round((passed_ex_df['End Score']['Lane Change'])*100), left_on='fullname', right_index=True)['Lane Change']




In [100]:
counts_df['Count']['Lane Change']

participant
Enoc Velazco Gallardo               10
Fernando Alejandro Rojas Barrios     7
Héctor Ruiz Flores                   7
Jesús Cruz González                  7
Juan Guerrero Ortiz                  6
Luis Alberto Zozaya Ordoñez          7
Luis Alejandro Novelo Ramírez        7
Luis Martín Lima González            7
Miguel Angel Ortiz Herrera           5
Name: Lane Change, dtype: int64

#### Setting the right company

In [101]:
for index, row in report_df.iterrows():
    for i, x in students.iterrows():
        if row.fullname == x.fullname:
            x = x.copy()
            if pd.isnull(x.company):
                company = raw_values.loc[0, 'Client']
            else:
                company = x.company
            report_df.loc[index, 'company'] = company
        else:
            pass

In [102]:
#Including Comments
for index, row in report_df.iterrows():
    for i, x in comments_x.iterrows():
        if row.fullname == x.fullname:
            x = x.copy()
            comment = x.comment
#             print(comment)
            report_df.loc[index, 'comments'] = comment
        else:
            pass

In [103]:
# report_df

In [104]:
finalx_df.loc[finalx_df['final_result'] < .8]

,exercise,participant,car_ID,stress,rev_slalom,rev_%,slalom,LnCh,cones,gates,f_time,final_result
2,Final Exercise,Héctor Ruiz Flores,1.0,0.0,0 days 00:00:19.450000,0.24,52,43,1,0,0 days 00:01:24.030000,0.78
3,Final Exercise,Jesús Cruz González,1.0,0.0,0 days 00:00:18.090000,0.22,70,60,4,0,0 days 00:01:29.420000,0.70
5,Final Exercise,Enoc Velazco Gallardo,1.0,0.0,0 days 00:00:17.950000,0.23,74,82,3,0,0 days 00:01:25.170000,0.77
6,Final Exercise,Luis Martín Lima González,1.0,0.0,0 days 00:00:22.190000,0.25,48,59,3,1,0 days 00:01:40.140000,0.55
9,Final Exercise,Luis Martín Lima González,1.0,1.0,0 days 00:00:17.400000,0.22,62,66,3,1,0 days 00:01:28.810000,0.71


In [105]:
# report_df

In [106]:
report_df_convert_dict = {
}

In [107]:
# report_df['prcnt_lc_pass'].astype(int)

#### Cambio Importante en como se leen las fechas

In [108]:
# Final Exercise Variables - Possible setting as table for multiple occurances
mse_report = pd.DataFrame(finalx_df.replace(np.nan, '-')
                          .drop(['exercise', 'rev_slalom'], axis=1)
                        )

mse_report['f_time'] = mse_report['f_time'].astype(str).str.extract('days 00:(.*\..{2}|.*$)') #<------New formula for slicing

if raw_values.loc[0, 'Country'] == 'MX':
    mse_report['stress'].replace((1, 0), ('Alto', 'Bajo'), inplace=True)
else:
    mse_report['stress'].replace((1, 0), ('High', 'Low'), inplace=True)

mse_report[['rev_%', 'final_result']] = mse_report[['rev_%', 'final_result']].apply(lambda x: x*100).astype(int)
mse_report.rename(columns={'rev_%' : 'rev_pc'}, inplace=True)

for i, r in mse_report.iterrows():
    if '.' in r.f_time:
        pass
    else:
        r = r.copy()
        ov = r.f_time
        ov = ov+".01"
        mse_report.loc[i, 'f_time'] = ov   #<------- Cambio nuevo

mse_report = mse_report.groupby('participant').apply(lambda x: x.to_dict(orient='records'))

## Jinja Variables Loading

In [109]:
tmplt_data = pd.read_excel('jinja_variables.xlsx', skiprows=2)
tmplt_context = dict(zip(tmplt_data['var'], tmplt_data['value']))

In [110]:
# for index, row in report_df.iterrows(): 
#     if row['fullname'] not in finalx_df['participant'].values:
#         print(row['fullname'] + ' Not in here')
#     else:
#         print('found')

In [111]:
report_df = report_df.fillna(0)

In [112]:
# finalx_df['final_result'] = finalx_df['final_result'].apply(lambda x: [y if y <= 1 else 0 for y in x])
finalx_df['final_result'] = [0 if x >= 1 else x for x in finalx_df['final_result']]

# finalx_df['final_result'] = 

C:\Users\pom\AppData\Local\Temp\ipykernel_4444\144853828.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalx_df['final_result'] = [0 if x >= 1 else x for x in finalx_df['final_result']]


In [113]:
finalx_df

,exercise,participant,car_ID,stress,rev_slalom,rev_%,slalom,LnCh,cones,gates,f_time,final_result
0,Final Exercise,Miguel Angel Ortiz Herrera,1.0,0.0,0 days 00:00:14.300000,0.20,71,74,1,0,0 days 00:01:12.330000,0.95
1,Final Exercise,Luis Alberto Zozaya Ordoñez,1.0,0.0,0 days 00:00:15.810000,0.21,61,54,0,0,0 days 00:01:16,0.90
2,Final Exercise,Héctor Ruiz Flores,1.0,0.0,0 days 00:00:19.450000,0.24,52,43,1,0,0 days 00:01:24.030000,0.78
3,Final Exercise,Jesús Cruz González,1.0,0.0,0 days 00:00:18.090000,0.22,70,60,4,0,0 days 00:01:29.420000,0.70
4,Final Exercise,Juan Guerrero Ortiz,1.0,0.0,0 days 00:00:15.730000,0.21,72,54,4,0,0 days 00:01:21.390000,0.82
5,Final Exercise,Enoc Velazco Gallardo,1.0,0.0,0 days 00:00:17.950000,0.23,74,82,3,0,0 days 00:01:25.170000,0.77
6,Final Exercise,Luis Martín Lima González,1.0,0.0,0 days 00:00:22.190000,0.25,48,59,3,1,0 days 00:01:40.140000,0.55
7,Final Exercise,Fernando Alejandro Rojas Barrios,1.0,0.0,0 days 00:00:17.570000,0.23,73,46,0,0,0 days 00:01:15.670000,0.90
8,Final Exercise,Luis Alejandro Novelo Ramírez,1.0,0.0,0 days 00:00:15.900000,0.21,65,57,1,0,0 days 00:01:17.340000,0.88
9,Final Exercise,Luis Martín Lima González,1.0,1.0,0 days 00:00:17.400000,0.22,62,66,3,1,0 days 00:01:28.810000,0.71


## Final Report Creation

In [117]:
# Jinja function to assign variables

def create_report(data):
    i = 0
    global student, program, fulldate, vehicle, s_graph, snor, spoce, saoep, saovc, sfpl, l_graph, lnor, lpoce, laoep, laovc, lfpl, msed_rev_time, comment, items, pct_graph, mse_graph, slalom_graph, lnch_graph_1, rev_graph
    
    for index, row in data.iterrows():
        if i > len(data):
            break
        else:
            if raw_values.loc[0, 'Country'] == 'MX':
                doc = DocxTemplate('esp_performance_report_tmplt.docx')
            else:
                doc = DocxTemplate('eng_performance_report_tmplt.docx')
            
            #Define variables for template
            
            if row['fullname'] not in finalx_df['participant'].values:
                pass
            else:
                student = row['fullname']
            
            
            
            fullname = student
            company = row['company']
            program = row['program']
            
            # Date language Format
            if raw_values.loc[0, 'Country'] == 'MX':
                fulldate = row['date'].strftime("%d / %m / %Y")
            else:
                fulldate = row['date'].strftime("%B %d %Y")
            
            vehicle = row['vehicle']
            snor = row['s_no_runs']
            spoce = int(row['prcnt_s_pass'])
            saoep = int(row['avg_ex_control_s'])
            saovc = int(row['avg_v_control_s'])
            sfpl = int(row['slalom_max'])
            lnor = row['lc_no_runs']
            lpoce = int(row['prcnt_lc_pass'])
            laoep = int(row['avg_ex_control_lc'])
            laovc = int(row['avg_v_control_lc']) 
            lfpl = int(row['lnch_max'])
            
            #Graphs Variables
            ax1_slalom_plt = (course_df.loc[
                ((course_df['participant'] == student) & 
                 (course_df['exercise'] == 'Slalom')), '%_of_exercise'
            ].astype(float))*100
            ax1_slalom_plt = ax1_slalom_plt.reset_index(drop=True)
            ax2_slalom_plt = (course_df.loc[
                ((course_df['participant'] == student) & 
                 (course_df['exercise'] == 'Slalom')), '%_of_vehicle'
            ].astype(float))*100
            ax2_slalom_plt = ax2_slalom_plt.reset_index(drop=True)
            
            ax1_LnCh_plt = (course_df.loc[
                ((course_df['participant'] == student) & 
                 (course_df['exercise'] == 'Lane Change')), '%_of_exercise'
            ].astype(float))*100
            ax1_LnCh_plt = ax1_LnCh_plt.reset_index(drop=True)
            ax2_LnCh_plt = (course_df.loc[
                ((course_df['participant'] == student) & 
                 (course_df['exercise'] == 'Lane Change')), '%_of_vehicle'
            ].astype(float))*100
            ax2_LnCh_plt = ax2_LnCh_plt.reset_index(drop=True)
            
            # Instructor comments 
            paragraph = row['comments']
            
            #Slalom Runs Graph
            plt.style.use('seaborn-dark-palette')
            plt.figure(figsize=(6.5,2))
            plt.axhline(y=80, color='#C87867', ls='--', lw=3)
            plt.annotate('Ideal', (0, 90), ha='center', va='center', fontsize=8, color='#C87867')
            plt.axhline(y=100, color='#67BEC8', ls='--', lw=3)
            plt.annotate('Ideal', (0, 110), ha='center', va='center', fontsize=8, color='#67BEC8')
            
            
            # Language Specific
            if raw_values.loc[0, 'Country'] == 'MX':
                plt.plot(ax1_slalom_plt, label = '% Del Ejercicio', linewidth=2, color='#001EBA')
                plt.plot(ax2_slalom_plt, label = '% Del Vehículo', linewidth=2, color='#BA0000')
                plt.title('Resultados de Slalom - ' + student)
            else:
                plt.plot(ax1_slalom_plt, label = '% Of The Exercise', linewidth=2, color='#001EBA')
                plt.plot(ax2_slalom_plt, label = '% Of The Vehicle', linewidth=2, color='#BA0000')
                plt.title(student + ' ' + 'Slalom Results')
            
            plt.ylim(ymin=0, ymax=120)
            plt.legend()
            
            
              #Language specific labels
            if raw_values.loc[0, 'Country'] == 'MX':
                plt.xlabel('Intentos')
            else:
                plt.xlabel('Runs')
            
            plt.ylabel('%')
            plt.tight_layout()
            plt.savefig('plots/Slalom Graph ' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            
            s_graph = InlineImage(doc, 'plots/Slalom Graph ' + student + '.png')
            
            #Lane Change Graph
            plt.style.use('seaborn-dark-palette')
            plt.figure(figsize=(6.5,2))
            plt.axhline(y=80, color='#C87867', ls='--', lw=3)
            plt.annotate('Ideal', (0, 90), ha='center', va='center', fontsize=8, color='#C87867')
            plt.axhline(y=100, color='#67BEC8', ls='--', lw=3)
            plt.annotate('Ideal', (0, 110), ha='center', va='center', fontsize=8, color='#67BEC8')
            
               # Language Specific
            if raw_values.loc[0, 'Country'] == 'MX':
                plt.plot(ax1_LnCh_plt, color='#001EBA', label = '% Del Ejercicio', linewidth=2)
                plt.plot(ax2_LnCh_plt, color='#BA0000', label = '% Del Vehículo', linewidth=2)
                plt.title('Resultados de Evasión de Barricada - ' + student)
            else:
                plt.plot(ax1_LnCh_plt, color='#001EBA', label = '% Of The Exercise', linewidth=2)
                plt.plot(ax2_LnCh_plt, color='#BA0000', label = '% Of The Vehicle', linewidth=2)
                plt.title(student + ' ' + 'Barricade Evasion Results')
            
            plt.ylim(ymin=0, ymax=120)
            plt.legend()
            
            #Language specific labels
            if raw_values.loc[0, 'Country'] == 'MX':
                plt.xlabel('Intentos')
            else:
                plt.xlabel('Runs')
            
            plt.ylabel('%')
            plt.tight_layout()
            plt.savefig('plots/LnCh Graph ' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            
            l_graph = InlineImage(doc, 'plots/LnCh Graph ' + student + '.png')
            
            #Final exercise table population
            items = mse_report[student]
            
# Final Exercise Percetage Graph - NEW
            column_x = finalx_df.loc[(finalx_df['participant'] == student)]['final_result']
            row_x = column_x.idxmax()

            # Data Variables
            data_slalom = finalx_df.iloc[row_x]['slalom']
            data_lnch = finalx_df.iloc[row_x]['LnCh']
            data_rev = finalx_df.iloc[row_x]['rev_%']*100
            data_s = finalx_df.iloc[row_x]['final_result']*100
            min_std = 80 - data_s
            if min_std < 0:
                min_std = 0
            else:
                pass
            data_y = 100 - (min_std + data_s)
            size = [data_s, min_std, data_y]
            marker_a = [79,1,20]

            dif_pct = 100 - mseg_perf
            gp_pct = [mseg_perf, dif_pct]
            # min_std = 80
            min_std_df = 100 - min_std
            avg_size = [min_std, min_std_df]
            my_circle = plt.Circle((0,0), 0.7, color='white')
            
            # Colors for percetage graph
            if data_s >= 80:
                colors_x = ['lawngreen', 'gainsboro', 'grey']
            elif data_s >= 70 < 80:
                colors_x = ['gold', 'gainsboro', 'grey']
            elif data_s < 70:
                colors_x = ['darkred', 'gainsboro', 'grey']

            # Overall Perf Graph
            fig, ax = plt.subplots(figsize=(1.5,1.5))
            ax.axis('equal')
            
            ## Local Levels
            if raw_values.loc[0, 'Country'] == 'MX':
                label1 = """Min.80%"""
                label2 = "Desempeño"
            else:
                label1 = """Min.80%"""
                label2 = "Performance"

            pie_3_names = ['',label1, '']

            pie_back, _ = ax.pie(marker_a, radius = 1.3+.1, colors=['white', 'black', 'white'], labels=pie_3_names, counterclock = False, startangle=-90, textprops={'fontsize': 10})
            plt.setp(pie_back, width=0.3, edgecolor='white')
            pie_1, _ = ax.pie(size, radius = 1.3, colors=colors_x, counterclock = False, startangle=-90, textprops={'fontsize': 8})
            plt.setp(pie_1, width=0.4, edgecolor='white')
            ax.annotate((str(int(data_s)) + '%'), (0, 0), ha='center', va='center', fontsize=16, fontweight='bold', color='black')
#             ax.annotate((label2), (0, .35), ha='center', va='center', fontsize=8, color='black')
            plt.margins(0,0)
            plt.savefig('plots/final_percent-' + student + '.png', bbox_inches='tight', dpi=300)
#             plt.show()
            plt.close()
            
            pct_graph = InlineImage(doc, 'plots/final_percent-' + student + '.png')
            
#Slalom Gpraph
            min_std = 80 - data_slalom
            if min_std < 0:
                min_std = 0
            else:
                pass
            data_y_slalom = 100 - (min_std + data_slalom)
            size = [data_slalom, min_std, data_y_slalom]
            marker_a = [79,1,20]

            dif_pct = 100 - mseg_perf
            gp_pct = [mseg_perf, dif_pct]
            # min_std = 80
            min_std_df = 100 - min_std
            avg_size = [min_std, min_std_df]
            my_circle = plt.Circle((0,0), 0.7, color='white')

            fig, ax = plt.subplots(figsize=(1,1))
            ax.axis('equal')

            if data_slalom >= 80:
                colors_x = ['lawngreen', 'gainsboro', 'grey']
            elif data_slalom >= 70 < 80:
                colors_x = ['gold', 'gainsboro', 'grey']
            elif data_slalom < 70:
                colors_x = ['darkred', 'gainsboro', 'grey']

            ## Local Levels
            if raw_values.loc[0, 'Country'] == 'MX':
                label1 = """Min. 80%"""
                label3 = ""
            else:
                label1 = """Min. 80%"""
                label3 = ""

            pie_3_names = ['',label1, '']

            pie_back, _ = ax.pie(marker_a, 
                                 radius = 1.3+.1, 
                                 colors=['white', 'black', 'white'], 
                                 labels=pie_3_names, 
                                 counterclock = False, 
                                 startangle=-90,
                                 textprops={'fontsize': 8},
                                 rotatelabels = False,
                                 labeldistance = 1
                                )
            plt.setp(pie_back, width=0.3, edgecolor='white')
            pie_1, _ = ax.pie(size, radius = 1.3, colors=colors_x, counterclock = False, startangle=-90, textprops={'fontsize': 6})
            plt.setp(pie_1, width=0.5, edgecolor='white')
            ax.annotate((str(int(data_slalom)) + '%'), (0, 0), ha='center', va='center', fontsize=10, fontweight='bold', color='black')
            ax.annotate((label3), (0, .35), ha='center', va='center', fontsize=6, color='black')
            plt.margins(0,0)

            plt.savefig('plots/final_slalom_percent-' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close() 
            
            slalom_graph = InlineImage(doc, 'plots/final_slalom_percent-' + student + '.png') #Slalom Graph
            
#LaneChange Gpraph
            min_std = 80 - data_lnch
            if min_std < 0:
                min_std = 0
            else:
                pass
            data_y_lnch = 100 - (min_std + data_lnch)
            size = [data_lnch, min_std, data_y_lnch]
            marker_a = [79,1,20]

            dif_pct = 100 - mseg_perf
            gp_pct = [mseg_perf, dif_pct]
            # min_std = 80
            min_std_df = 100 - min_std
            avg_size = [min_std, min_std_df]
            my_circle = plt.Circle((0,0), 0.7, color='white')

            fig, ax = plt.subplots(figsize=(1,1))
            ax.axis('equal')

            if data_lnch >= 80:
                colors_x = ['lawngreen', 'gainsboro', 'grey']
            elif data_lnch >= 70 < 80:
                colors_x = ['gold', 'gainsboro', 'grey']
            elif data_lnch < 70:
                colors_x = ['darkred', 'gainsboro', 'grey']

            ## Local Levels
            if raw_values.loc[0, 'Country'] == 'MX':
                label1 = """Min. 80%"""
                label4 = ""
            else:
                label1 = """Min. 80%"""
                label4 = ""

            pie_3_names = ['',label1, '']

            pie_back, _ = ax.pie(marker_a, 
                                 radius = 1.3+.1, 
                                 colors=['white', 'black', 'white'], 
                                 labels=pie_3_names, 
                                 counterclock = False, 
                                 startangle=-90,
                                 textprops={'fontsize': 8},
                                 rotatelabels = False,
                                 labeldistance = 1
                                )
            plt.setp(pie_back, width=0.3, edgecolor='white')
            pie_1, _ = ax.pie(size, radius = 1.3, colors=colors_x, counterclock = False, startangle=-90, textprops={'fontsize': 6})
            plt.setp(pie_1, width=0.5, edgecolor='white')
            ax.annotate((str(int(data_lnch)) + '%'), (0, 0), ha='center', va='center', fontsize=10, fontweight='bold', color='black')
            ax.annotate((label4), (0, .35), ha='center', va='center', fontsize=6, color='black')
            plt.margins(0,0)

            plt.savefig('plots/final_lnch_percent-' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            
            lnch_graph_1 = InlineImage(doc, 'plots/final_lnch_percent-' + student + '.png') #LaneChange Gpraph
            
#Reverse Gpraph
            min_std = 20 - data_rev
            if min_std < 0:
                min_std = 0
            else:
                pass
            data_y_rv = 100 - (min_std + data_rev)
            size = [data_rev, min_std, data_y_rv]
            marker_a = [19,1,80]

            dif_pct = 100 - mseg_perf
            gp_pct = [mseg_perf, dif_pct]
            # min_std = 80
            min_std_df = 100 - min_std
            avg_size = [min_std, min_std_df]
            my_circle = plt.Circle((0,0), 0.7, color='white')

            fig, ax = plt.subplots(figsize=(1,1))
            ax.axis('equal')

            if data_rev > 25:
                colors_x = ['darkred', 'gainsboro', 'grey']
            elif data_rev > 20 <= 25:
                colors_x = ['gold', 'gainsboro', 'grey']
            elif data_rev <= 20:
                colors_x = ['lawngreen', 'gainsboro', 'grey']

            ## Local Levels
            if raw_values.loc[0, 'Country'] == 'MX':
                label1 = """Max. 20%"""
                label5 = ""
            else:
                label1 = """Max. 20%"""
                label5 = ""

            pie_3_names = ['',label1, '']

            pie_back, _ = ax.pie(marker_a, 
                                 radius = 1.3+.1, 
                                 colors=['white', 'black', 'white'], 
                                 labels=pie_3_names, 
                                 counterclock = False, 
                                 startangle=-90, 
                                 textprops={'fontsize': 8},
                                 rotatelabels = False,
                                 labeldistance = 1
                                )
#             print(student, size)
            plt.setp(pie_back, width=0.3, edgecolor='white')
            pie_1, _ = ax.pie(size, radius = 1.3, colors=colors_x, counterclock = False, startangle=-90, textprops={'fontsize': 6})
            plt.setp(pie_1, width=0.5, edgecolor='white')
            ax.annotate((str(int(data_rev)) + '%'), (0, 0), ha='center', va='center', fontsize=10, fontweight='bold', color='black')
            ax.annotate((label5), (0, .35), ha='center', va='center', fontsize=6, color='black')
            plt.margins(0,0)

            plt.savefig('plots/final_rv_percent-' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            
            rev_graph = InlineImage(doc, 'plots/final_rv_percent-' + student + '.png') #Reverse Gpraph
            
# Final Exercise Graph -New


            data_time = finalx_df.loc[finalx_df['participant'] == student].f_time.dt.total_seconds()
            data_time = data_time.reset_index(drop=True)
            data_performance = finalx_df.loc[finalx_df['participant'] == student].final_result*100
            data_performance = data_performance.reset_index(drop=True)
            data_rv = finalx_df.loc[finalx_df['participant'] == student].rev_slalom.dt.total_seconds()
            data_rv = data_rv.reset_index(drop=True)
            
            
            data_tm_fn = data_performance
            cones_fn = finalx_df.loc[finalx_df['participant'] == student].cones
            cones_fn = cones_fn.reset_index(drop=True)
            gates_fn = finalx_df.loc[finalx_df['participant'] == student].gates
            gates_fn = gates_fn.reset_index(drop=True)
            xtick = mse_report[student][0]['f_time']
            cone_img = mpimg.imread('assets/cones_for_reports.png')

            av_rev_result = finalx_df['rev_slalom'].mean().total_seconds()
            av_gp_time = finalx_df['f_time'].mean().total_seconds()

            pass_time = 80 #(ideal_time.total_seconds()*1.2)
            
            # New Values for Graphs #<-----Changed
            data_rv_pct = (data_rv / data_time)*100
            data_per_fn = data_performance - data_rv_pct

            slalom_pct = finalx_df.loc[finalx_df['participant'] == student].slalom
            slalom_pct = slalom_pct.reset_index(drop=True)
            lnch_pct = finalx_df.loc[finalx_df['participant'] == student].LnCh
            lnch_pct = lnch_pct.reset_index(drop=True)

            passes = data_time.index.tolist()
            passes.append(2)

            label = finalx_df.loc[finalx_df['participant'] == student].stress
            if raw_values.loc[0, 'Country'] == 'MX':
                label = label.reset_index(drop=True).replace(0, "Bajo Estrés").replace(1, 'Alto Estrés')
            else:
                label = label.reset_index(drop=True).replace(0, "Low Stress").replace(1, 'High Stress')


            data_sx = mse_report[student][0]['final_result']
            min_stdx = 80 - data_sx
            if min_stdx < 0:
                min_stdx = 0
            else:
                pass
            data_yx = 100 - (min_std + data_s)

            barHeight = .8

            fig, ax = plt.subplots(figsize=(8,4))
            
            ax.invert_yaxis()
            
            imagebox = OffsetImage(cone_img, zoom=0.1)
            imagebox.image.axes = ax

            # Average Bar
            ## labels
            if raw_values.loc[0, 'Country'] == 'MX':
                label3 = """Promedio: """
                label4 = 'Prom. Rev: '
                label5 = """ Prom. Penalización: """
                label6 = 'Prom. Gpo. '
            else:
                label3 = """Average: """
                label4 = 'Av. Rev: '
                label5 = """ Av. Penalties: """
                label6 = 'Gp. Average '
            
            ax.barh('Av', float(mseg_rev), color='khaki', edgecolor='white', height=.3) #<-----Changed
            ax.barh('Av', float(mseg_perf)-float(mseg_rev), left=float(mseg_rev), color='gold', edgecolor='white', height=.3)#<-----Changed
            
            ax.annotate(label3 + str(mseg_t), (mseg_perf, .3), color='black', va='center', ha='right', fontsize=8)
            ax.annotate(label4 + str(mseg_rev), (2, .3), color='black', va='center', ha='left', fontsize=8)
#             ax.annotate(label5 + str(mseg_c) + " | " + str(mseg_g), (mseg_perf, .3), color='black', fontsize=8, va='center', ha='center')
            ax.annotate(label5 + str(mseg_c) + " | " + str(mseg_g), (float(mseg_rev) + 10, .3), color='black', fontsize=8, va='center', ha='left')
            ax.annotate(label6 + str(mseg_perf) + '%', xy=(float(mseg_perf) + .3, 0), xytext=(float(mseg_perf) + 10, 0), ha='left', fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')

            # Main Bar Plot
            ## Labels ##
            if raw_values.loc[0, 'Country'] == 'MX':
                label7 = """T: """ 
                label8 = """Aceptable"""
                label9 = """Penalizaciones: """
            else:
                label7 = """T: """ 
                label8 = """Acceptable"""
                label9 = """Penalties: """

            for i, v in data_tm_fn.items():
            
            # Student Bar
                ax.barh(i+1, data_rv_pct[i], 
                        color='darkred', 
                        edgecolor='white', 
                        height=barHeight) 
                ax.barh(i+1, v - data_rv_pct[i], 
                        left=data_rv_pct[i], 
                        color='red', 
                        edgecolor='white', 
                        height=barHeight) 

                xtick1 = (mse_report[student][i]['f_time'])
                xtick2 = xtick1 #str(datetime.strptime(xtick1, '%M:%S.%f').time()) <------Deprecated (No se necesita con el cambio anterior)
                
                # Time Annotations
                ax.annotate(label7 + xtick2, 
                            (v - 1, i+1), 
                            color='w', fontsize=10, 
                            fontweight='bold', 
                            va='center', 
                            ha='right')
                ax.annotate(str(int(data_performance[i]))+'%', 
                            (v + 1, i+1), 
                            color='black', 
                            fontsize=10, 
                            fontweight='bold', 
                            va='center', 
                            ha='left')

            # Acceptable Reverse Annotation
#                 ax.annotate('', (.1, i + 1.4), xytext=((80 * .2), i + 1.4), #<------ Hay un error en esta formula, no marca la realidad
#                             arrowprops=dict(arrowstyle='|-|'), 
#                             va='top')
#                 ax.annotate(label8,(1, i + 1.5), 
#                             xytext=(80 * .10, i + 1.5), 
#                             va='center', ha='center', fontsize=8)

            # Stress Labels
                if label[i] == "High Stress" or "Alto Estrés":
                    label_color = 'orangered'
                else:
                    label_color = 'yellowgreen'
                
                ax.annotate(label[i], 
                            ((v)+1, i + .8), 
                            ha='left', va='top', 
                            fontsize=8, fontweight='bold', 
                            color=(label_color))
                
                
                # Add Penalties Image
                ab = AnnotationBbox(imagebox, 
                                    ((data_performance[i] - data_rv_pct[i]) - 11, i+1.4),     # Bar Image 
                                    xybox=(0., 0.), 
                                    xycoords='data',
                                    boxcoords="offset points", 
                                    frameon=False)
                ax.add_artist(ab)
                
                # Penalties Cones | Gates
                ax.annotate(label9 + str(cones_fn[i]) + " | " + str(gates_fn[i]), 
                            (data_performance[i] - data_rv_pct[i], i+1.5), 
                            color='black', 
                            fontsize=8, 
                            ha='center', va='center')
                
                # Slalom | LnCh Percetage Annotation
                if data_performance[i] < 60:
                    label10 = 'S' 
                    label11 = 'L'
                    labels_offset = .8
                else:
                    label10 = 'Slalom'
                    label11 = 'LnCh'
                    labels_offset = 1
                ax.annotate(label10 + ' ' + str(slalom_pct[i]) + '%' + " | " + label11 + ' ' + str(lnch_pct[i]) + '%', 
                            (data_rv_pct[i] + 5, i + labels_offset), 
                            color='w', 
                            fontsize = 10, fontweight='bold', 
                            va='center', ha='left')

            
            # Reverse Time Annotation
            for i, v in data_rv.items():
                if raw_values.loc[0, 'Country'] == 'MX':
                    ax.annotate("Reversa:" + str(round(float(v),2)), (2, i+1), color='w', fontsize=10, fontweight='bold', va='center') #Reverse Bar Legend
                else:
                    ax.annotate("Reverse:" + str(round(float(v),2)), (2, i+1), color='w', fontsize=10, fontweight='bold', va='center') #Reverse Bar Legend

            # Pass or Fail Annotation <-----Changed
            ax.axvline(pass_time, linestyle='--', ymax=len(passes), color='aqua')
            if raw_values.loc[0, 'Country'] == 'MX':
                ax.annotate('Necesita Práctica', xy=(pass_time - 23, len(passes)-.3), xytext=(pass_time -1, len(passes)-.3), ha='right', fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')
                ax.annotate('Security Driver', xy=(pass_time + 20, len(passes)-.3), xytext=(pass_time +1, len(passes)-.3), fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')
            else:
                ax.annotate('Needs More Work', xy=(pass_time - 30, len(passes)-.3), xytext=(pass_time -1, len(passes)-.3), ha='right', fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')
                ax.annotate('Security Driver', xy=(pass_time + 20, len(passes)-.3), xytext=(pass_time +1, len(passes)-.3), fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')

            ax.set_yticks([])
            ax.set_xticks([])
            ax.axis('off')

            fig.tight_layout()
            plt.xlim(xmin=0, xmax=100)
            plt.ylim(ymin=-.5, ymax=len(passes))
            plt.savefig('plots/final_exercise-' + student + '.png', bbox_inches='tight', dpi=300)
#             plt.show()
            plt.close()
            
            mse_graph = InlineImage(doc, 'plots/final_exercise-' + student + '.png')
            
            for var in tmplt_context:
                tmplt_context[var] = eval(var)
            
            jinja_env = jinja2.Environment(autoescape=True)
            doc.render(tmplt_context, jinja_env)
            doc.save('reports/' + str(student) + ' - Report.docx')
            
            comment = str(paragraph)
            
            i+=1
            
            

In [115]:
(course_df.loc[
                ((course_df['participant'] == student) & 
                 (course_df['exercise'] == 'Slalom')), '%_of_exercise'
            ].astype(float))*100

NameError: name 'student' is not defined

In [115]:
create_report(report_df)

C:\Users\pom\AppData\Local\Temp\ipykernel_4444\3317746092.py:592: MatplotlibDeprecationWarning: Support for passing numbers through unit converters is deprecated since 3.5 and support will be removed two minor releases later; use Axis.convert_units instead.
  ax.set_yticks([])
C:\Users\pom\AppData\Local\Temp\ipykernel_4444\3317746092.py:592: MatplotlibDeprecationWarning: Support for passing numbers through unit converters is deprecated since 3.5 and support will be removed two minor releases later; use Axis.convert_units instead.
  ax.set_yticks([])
C:\Users\pom\AppData\Local\Temp\ipykernel_4444\3317746092.py:592: MatplotlibDeprecationWarning: Support for passing numbers through unit converters is deprecated since 3.5 and support will be removed two minor releases later; use Axis.convert_units instead.
  ax.set_yticks([])
C:\Users\pom\AppData\Local\Temp\ipykernel_4444\3317746092.py:592: MatplotlibDeprecationWarning: Support for passing numbers through unit converters is deprecated sinc

In [116]:
ev_date = raw_values['Date'][0].strftime("%b-%d-%y")

In [117]:
# course_df.to_csv('reports/Course_df - ' + company + ' - ' + ev_date + '.csv')
# finalx_df.to_csv('reports/Finalx_df - ' + company + ' - ' + ev_date + '.csv')
# raw_values.to_csv('reports/Raw_values - ' + company + ' - ' + ev_date + '.csv')

In [118]:
finalx_df

,exercise,participant,car_ID,stress,rev_slalom,rev_%,slalom,LnCh,cones,gates,f_time,final_result
0,Final Exercise,Miguel Angel Ortiz Herrera,1.0,0.0,0 days 00:00:14.300000,0.20,71,74,1,0,0 days 00:01:12.330000,0.95
1,Final Exercise,Luis Alberto Zozaya Ordoñez,1.0,0.0,0 days 00:00:15.810000,0.21,61,54,0,0,0 days 00:01:16,0.90
2,Final Exercise,Héctor Ruiz Flores,1.0,0.0,0 days 00:00:19.450000,0.24,52,43,1,0,0 days 00:01:24.030000,0.78
3,Final Exercise,Jesús Cruz González,1.0,0.0,0 days 00:00:18.090000,0.22,70,60,4,0,0 days 00:01:29.420000,0.70
4,Final Exercise,Juan Guerrero Ortiz,1.0,0.0,0 days 00:00:15.730000,0.21,72,54,4,0,0 days 00:01:21.390000,0.82
5,Final Exercise,Enoc Velazco Gallardo,1.0,0.0,0 days 00:00:17.950000,0.23,74,82,3,0,0 days 00:01:25.170000,0.77
6,Final Exercise,Luis Martín Lima González,1.0,0.0,0 days 00:00:22.190000,0.25,48,59,3,1,0 days 00:01:40.140000,0.55
7,Final Exercise,Fernando Alejandro Rojas Barrios,1.0,0.0,0 days 00:00:17.570000,0.23,73,46,0,0,0 days 00:01:15.670000,0.90
8,Final Exercise,Luis Alejandro Novelo Ramírez,1.0,0.0,0 days 00:00:15.900000,0.21,65,57,1,0,0 days 00:01:17.340000,0.88
9,Final Exercise,Luis Martín Lima González,1.0,1.0,0 days 00:00:17.400000,0.22,62,66,3,1,0 days 00:01:28.810000,0.71


In [119]:
# convert('reports/') #<--- Convert to PDF

# New ReportLab report

In [120]:
'-------------------------------------------------------------'
'Nex Table For Final Exercise'
'-------------------------------------------------------------'

mse_student = finalx_df[finalx_df['participant'] == student].index.tolist()
# finalx_df_slice = finalx_df.iloc[7]
mse_data = pd.DataFrame(finalx_df.iloc[mse_student], columns=['stress', 'f_time', 'rev_%', 'slalom', 'LnCh', 'cones', 'gates', 'final_result'])
mse_data[['rev_%', 'final_result']] = (mse_data[['rev_%', 'final_result']]*100).astype(int)
mse_data['f_time'] = mse_data['f_time'].astype(str).str.extract('days 00:(.*\..{2}|.*$)')
if raw_values.loc[0, 'Country'] == 'MX':
    mse_data['stress'].replace((1, 0), ('Alto', 'Bajo'), inplace=True)
else:
    mse_data['stress'].replace((1, 0), ('High', 'Low'), inplace=True)
mse_data['rev_%'] = mse_data['rev_%'].astype(str) + '%'
mse_data['slalom'] = mse_data['slalom'].astype(str) + '%'
mse_data['LnCh'] = mse_data['LnCh'].astype(str) + '%'
mse_data['final_result'] = mse_data['final_result'].astype(str) + '%'

In [121]:
mse_data

,stress,f_time,rev_%,slalom,LnCh,cones,gates,final_result
2,Bajo,01:24.03,24%,52%,43%,1,0,78%
10,Alto,01:13.43,20%,59%,63%,0,0,94%


In [122]:
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.styles import ParagraphStyle as PS
from reportlab.platypus import PageBreak, Table, NextPageTemplate, Image, Spacer, Flowable, TableStyle
from reportlab.platypus.paragraph import Paragraph
from reportlab.platypus.doctemplate import PageTemplate, BaseDocTemplate
from reportlab.pdfbase.pdfmetrics import registerFont, registerFontFamily
from reportlab.platypus.tableofcontents import TableOfContents
from reportlab.platypus.frames import Frame
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.units import cm, inch
from reportlab.lib import colors, utils
from functools import partial
from reportlab.lib.pagesizes import letter
from PIL import Image as pImage

page_size = letter
pwidth, pheight = letter
fwidth = pwidth*.8

'-----------------------------------------------------------------------------'
'Register Fonts'
'-----------------------------------------------------------------------------'

registerFont(TTFont('MontserratBold', 'C:\\Users\\pom\\Creative Cloud Files\\Fonts\\Montserrat\\Montserrat ExtraBold 800.ttf'))
registerFont(TTFont('MontserratBlack', 'C:\\Users\\pom\\Creative Cloud Files\\Fonts\\Montserrat\\Montserrat Black 900.ttf'))
registerFont(TTFont('Montserrat', 'C:\\Users\\pom\\Creative Cloud Files\\Fonts\\Montserrat\\Montserrat-Regular.ttf'))
registerFont(TTFont('MontserratLight', 'C:\\Users\\pom\\Creative Cloud Files\\Fonts\\Montserrat\\Montserrat Light 300.ttf'))
registerFont(TTFont('MontserratThin', 'C:\\Users\\pom\\Creative Cloud Files\\Fonts\\Montserrat\\Montserrat Thin 250.ttf'))

from reportlab.lib.fonts import addMapping
# addMapping('SabonRom', 0, 0, 'SabonRom') #normal
# addMapping('SabonRom', 0, 1, 'SabonIta') #italic
addMapping('MontserratLight', 1, 0, 'MontserratBold') #bold
# addMapping('SabonRom', 1, 1, 'SabonBolIta') #italic and bold
# heavy = ParagraphStyle(name='normal', fontName='SabonRom', fontSize=10, leading=1.4*10 )


'-----------------------------------------------------------------------------'
'Template'
'-----------------------------------------------------------------------------'

class MyDocTemplate(BaseDocTemplate):

    def __init__(self, filename, **kw):
        self.allowSplitting = 0
        BaseDocTemplate.__init__(self, filename, **kw)

        template_CoverPage = PageTemplate('CoverPage',
            [Frame(.8*cm, pheight/2-(inch*3), pwidth/2, 230, id='F1', showBoundary=0)], 
            onPage=createCover, 
            )
        
        template_NormalPage = PageTemplate('NormalPage',
                                [Frame(pwidth*.08, 2.5*cm, pwidth*.84, pheight*.8, id='F2', 
                                    showBoundary=0)],
                                onPage=partial(header_and_footer,
                                header_content=header_content,
                                footer_content=footer_content),
                                pagesize=page_size)
        
        template_SecondPage = PageTemplate('SecondPage',
                                [Frame(pwidth*.08, pheight*.37, pwidth*.84, pheight*.50, id='F3', 
                                    showBoundary=0),
                                Frame(pwidth*.08, 3*cm, pwidth*.42, pheight*.25, id='F4', 
                                    showBoundary=0),
                                Frame(pwidth*.50, 3*cm, pwidth*.42, pheight*.25, id='F5', 
                                    showBoundary=0)],
                                onPage=partial(header_and_footer,
                                header_content=header_content,
                                footer_content=footer_content),
                                pagesize=page_size)

        self.addPageTemplates([template_CoverPage,template_NormalPage, template_SecondPage])



    def afterFlowable(self, flowable):
        "Registers TOC entries."
        if flowable.__class__.__name__ == 'Paragraph':
            text = flowable.getPlainText()
            style = flowable.style.name
            if style == 'Heading1':
                self.notify('TOCEntry', (0, text, self.page))
            if style == 'Heading2':
                self.notify('TOCEntry', (1, text, self.page))

h1 = PS(name = 'Heading1',
       fontSize = 18,
       leading = 20,
       fontName='MontserratBold',
       textColor="#C10230",
       spaceAfter = 8,
       spaceBefore = 8)

h2 = PS(name = 'Heading2',
       fontSize = 14,
       fontName='MontserratBlack',
       leading = 16,
       leftIndent = 0)
h3 = PS(name = 'Heading3',
       fontSize = 14,
       fontName='MontserratBold',
       textColor="#C10230",
       leading = 16,
       leftIndent = 10)
subh2 = PS(name = 'SubHeading2',
       fontSize = 14,
       fontName='Montserrat',
       leading = 16,
       leftIndent = 5)
l0 = PS(name = 'list0',
        fontSize = 12,
        leading =14,
        leftIndent=0,
        rightIndent=0,
        spaceBefore = 12,
        spaceAfter =0
        )
pstyle = PS(name='ms',
        fontName='MontserratLight',
        fontSize=10,
        leading =14,
        spaceAfter = 8,
        spaceBefore = 8
        )
no_spacing = PS(name='ms',
        fontName='MontserratLight',
        fontSize=10,
        leading =14,
        spaceAfter = 0,
        spaceBefore = 0
        )
footertxt = PS(name='footer',
        fontName='MontserratLight',
        fontSize=10,
        leading =9,
        leftIndent=40,
        rightIndent=0,
        )
super_h1 = PS(name = 'Super_Heading1',
       fontSize = 80,
       leading = 0,
       fontName='MontserratBold',
       textColor="#C10230",
       spaceAfter = 0,
       spaceBefore = 0)
ttl = PS(name = 'Title',
       fontSize = 24,
       leading = 25,
       fontName='MontserratBlack',
       textColor="#000000",
       spaceAfter = 0,
       spaceBefore = 0)

'-----------------------------------------------------------------------------'
'Table Styles'
'-----------------------------------------------------------------------------'

normalTable = TableStyle([
    ('FONTNAME', (0,0), (0, -1), 'Montserrat'),
    ('FONTNAME', (0,-1), (-1,-1), 'Montserrat'),
    ('FONTNAME', (0,1), (-1,-2), 'MontserratLight'),
#     ('GRID',(0,0),(-1,-1),.5,colors.black),
    ('ALIGN', (0,0), (-1,0), 'CENTER'),
    ('ALIGN', (1,1), (-1,-1), 'CENTER'),
    ('LINEBELOW', (0,0), (-1,0), 2, colors.black),
    ('LINEABOVE', (0,-1), (-1,-1), .5, colors.black),
    ('LINEBELOW', (0,-1), (-1,-1), 2, colors.black),
])

mseTable = TableStyle([
    ('FONTNAME', (0,0), (0, -1), 'Montserrat'),
    ('FONTNAME', (0,-1), (-1,-1), 'Montserrat'),
    ('FONTNAME', (0,1), (-1,-2), 'MontserratLight'),
#     ('GRID',(0,0),(-1,-1),.5,colors.black),
    ('ALIGN', (1,0), (-1,0), 'CENTER'),
    ('ALIGN', (1,1), (-1,-1), 'CENTER'),
    ('LINEBELOW', (0,0), (-1,0), 2, colors.black),
    ('LINEABOVE', (0,-1), (-1,-1), .5, colors.black),
    ('LINEBELOW', (0,-1), (-1,-1), 2, colors.black),
    ('BACKGROUND', (0, -1), (-1,-1), colors.lightcoral),
    ('VALIGN', (0, -1), (-1,-1), 'MIDDLE'),
    
])

introTable = TableStyle([
    ('FONTNAME', (0,0), (-1, -1), 'Montserrat'),
#     ('GRID',(0,0),(-1,-1),.5,colors.black),
    ('ALIGN', (0,0), (-1,-1), 'LEFT'),
    ('LINEBELOW', (0,0), (-1,-1), 1, colors.black),
])

consoleTable = TableStyle([
    ('BOX',(0,0),(-1,-1), 2, colors.firebrick),
    ('ALIGN', (0,0), (0,-1), 'RIGHT'),
    ('ALIGN', (0,1), (-1,-1), 'CENTER'),
    ('VALIGN', (0,1), (-1,-1), 'TOP'),
    ('BACKGROUND', (0, 0), (-1,-1), colors.beige),
#     ('LINEBELOW', (0,0), (-1,-1), 1, colors.black),
])


subTable = TableStyle([
    ('FONTNAME', (0,0), (-1,-1), 'MontserratLight'),
#     ('GRID',(0,0),(-1,-1),.5,colors.black),
    ('VALIGN', (0,0), (-1,-1), 'TOP'),
    ('ALIGN', (0,0), (-1,-1), 'RIGHT'),
    ('TOPPADDING', (0,0), (0,0), 10)
])

'-----------------------------------------------------------------------------'
'Header and Footer'
'-----------------------------------------------------------------------------'
def header(canvas, doc, content):
    canvas.saveState()
    w, h = content.wrap(doc.width, doc.topMargin)
    content.drawOn(canvas, doc.leftMargin-22.4, doc.height + doc.bottomMargin + doc.topMargin - h -10)
    canvas.restoreState()

def footer(canvas, doc, content):
    drawPageNumber(canvas, doc)
    canvas.saveState()
    w, h = content.wrap(doc.width, doc.bottomMargin)
    content.drawOn(canvas, doc.leftMargin-22.4, h)
    canvas.restoreState()

def header_and_footer(canvas, doc, header_content, footer_content):
    header(canvas, doc, header_content)
    footer(canvas, doc, footer_content)

def drawPageNumber(canvas, doc):
    pageNumber = canvas.getPageNumber()
    # canvas.setFont("Helvetica",11)
    # canvas.drawCentredString(17.4*cm, 1.35*cm, 'Page '+str(pageNumber))

def PageNumber(canvas, doc):
    return(canvas.getPageNumber())

header_center_text='header_center_text foo'
footer_center_text='footer_center_text bar' 

'-----------------------------------------------------------------------------'
'Variables and Texts'
'-----------------------------------------------------------------------------'

logo_int = 'C:\\Users\\pom\\Creative Cloud Files\\Designs AS3 Drving\\AS3 Logos\\AS3 International\\Images\\Logo---AS3-international-200x200.png'
ceo_cfo = 'D:\\OneDrive - AS3 Driver Training\\AS3 International\\Designs - AS3 Int\\Other Assets\\images\\The-CFO-to-the-CEO---Logo.png'
ceo_cfo_inv = 'D:\\OneDrive - AS3 Driver Training\\AS3 International\\Designs - AS3 Int\\Other Assets\\images\\The-CFO-to-the-CEO---Logo_inverted.png'
as3_logo = 'C:\\Users\\pom\\Creative Cloud Files\\Designs AS3 Drving\\AS3 Logos\\Images\\AS3 Driver Training - Logo.png'
chrysler300 = 'D:\\OneDrive - AS3 Driver Training\\Python Projects\\as3_reports\\python_tests\\chrysler_300.png'

if raw_values.loc[0, 'Country'] == 'MX':
    with open('texts/[es]_personal_report_intro.txt', 'r', encoding="utf-8") as file:
        intro_pre = file.read()
        intro_text = intro_pre
    
    with open('texts/[es]slalom_description.txt', 'r', encoding="utf-8") as file:
        slalom_pre = file.read()
        slalom_text = slalom_pre

    with open('texts/[es]final_x_description.txt', 'r', encoding="utf-8") as file:
        mse_pre = file.read()
        mse_text = mse_pre
        
    with open('texts/[es]final_x_intro.txt', 'r', encoding="utf-8") as file:
        mse_pre2 = file.read()
        mse_desc = mse_pre2
    
    with open('texts/[es]final_note.txt', 'r', encoding="utf-8") as file:
        fn_note_pre = file.read()
        fn_note = fn_note_pre
    
    mainTitle = 'EVALUACIÓN POST ENTRENAMIENTO (AAR)'
    scndTitle = '<para alignment="center">EVALUACIÓN DE DESEMPEÑO</para>'
    thirdTitle = 'DESEMPEÑO EN EJERCICIOS'
    thirdSubTitle = 'EJERCICIOS DE DESARROLLO DE HABILIDADES'
    fourthSubTitle = 'EJERCICIO FINAL CONBINADO'
    slalomTitle = '<para><b>SLALOM</b> - 4 Conos, Cuerda 13mts</para>'
    LnChTitle = '<para><b>CAMBIO EVASIVO DE CARRIL</b> - Diseño Estándar, Cuerda de 32 mts</para>'
    mse_title = 'EJERCICIO MULTIDISCIPLINARIO – 9 Obstáculos, emboscada fija / en movimiento.'
    mse_consoleTitle = "KPI Personal - Mejor Resultado"
    inst_comments_title = 'COMENTARIOS DEL INSTRUCTOR LIDER'
    stdt_name = '<para><b><i>Nombre del Estudiante</i></b></para>'
    company_label = '<para><b><i>Empresa</i></b></para>'
    pgm_label = '<para><b><i>Programa</i></b></para>'
    pgmDt_label = '<para><b><i>Fecha del Programa</i></b></para>'
    vhcl_label = '<para><b><i>Vehículo</i></b></para>'
    intro_title = 'DESCRIPCIÓN DEL PROGRAMA'
    perf_table_title = 'DESEMPEÑO EN NÚMEROS'
    perf_table_heading = 'Promedio del Grupo'
    perf_table_runs = 'Número de Intentos'
    perf_table_completed = 'Porcentaje de Ejercicios Completados'
    perf_table_percetage = 'Promedio de Porcentaje del Ejercicio'
    perf_table_control = 'Promedio de Control del Vehículo'
    perf_table_maxControl = 'Nivel Máximo de Control Alcanzado'
    mse_barricade = '<para><font fontname="MontserratBold" color="darkred" size="14">Barricada</font></para>'
    mse_slalom = '<para><font fontname="MontserratBold" color="darkred" size="14">Slalom</font></para>'
    mse_reverse = '<para><font fontname="MontserratBold" color="darkred" size="14">Reversa</font></para>'
    mse_ova = '<para><font fontname="MontserratBold" color="darkred" size="14">Desempeño Global</font></para>'
    mse_graphTitle = 'Gráfica de Desempeño'
    mseTable_headers = [['Estrés', 'Tiempo Final', 'Reversa', 'Slalom', 'Barricada', Paragraph('<para align="center">Conos</para>'), Paragraph('<para align="center">Puertas</para>'), Paragraph('<para align="center">Resultado<br/> Final</para>')]]
    gpavg_lvl = 'Promedio del Grupo'

else:
    with open('texts/[en]_personal_report_intro.txt', 'r', encoding="utf-8") as file:
        intro_pre = file.read()
        intro_text = intro_pre
    
    with open('texts/[en]slalom_description.txt', 'r', encoding="utf-8") as file:
        slalom_pre = file.read()
        slalom_text = slalom_pre

    with open('texts/[en]final_x_description.txt', 'r', encoding="utf-8") as file:
        mse_pre = file.read()
        mse_text = mse_pre
        
    with open('texts/[en]final_x_intro.txt', 'r', encoding="utf-8") as file:
        mse_pre2 = file.read()
        mse_desc = mse_pre2
    
    with open('texts/[en]final_note.txt', 'r', encoding="utf-8") as file:
        fn_note_pre = file.read()
        fn_note = fn_note_pre
    
    mainTitle = 'AFER ACTION REPORT (AAR)'
    scndTitle = '<para alignment="center">PERFORMANCE EVALUATION</para>'
    thirdTitle = 'EXERCISE PERFORMANCE'
    thirdSubTitle = 'SKILL DEVELOPMENT EXERCISES'
    fourthSubTitle = 'COMBINED FINAL EXERCISE'
    slalomTitle = '<para><b>SLALOM</b> - 4 Cones, 50ft Chord</para>'
    LnChTitle = '<para><b>EVASIVE LANE CHANGE</b> - Standard Design, 100ft Chord</para>'
    mse_title = 'MULTI-DISCIPLINARY EXERCISE – 9 Obstacles, fixed / rolling ambush.'
    mse_consoleTitle = "Personal KPI - Top Result"
    inst_comments_title = 'LEAD INSTRUCTOR FEEDBACK'
    stdt_name = '<para><b><i>Student Name</i></b></para>'
    company_label = '<para><b><i>Company</i></b></para>'
    pgm_label = '<para><b><i>Program</i></b></para>'
    pgmDt_label = '<para><b><i>Program Date</i></b></para>'
    vhcl_label = '<para><b><i>Vehicle</i></b></para>'
    intro_title = 'PROGRAM DESCRIPTION'
    perf_table_title = 'PERFORMANCE IN NUMBERS'
    perf_table_heading = 'Group Average'
    perf_table_runs = 'Number of Runs'
    perf_table_completed = 'Percentage of Completed Exercises'
    perf_table_percetage = 'Average of Exercise Percentage'
    perf_table_control = 'Average of Vehicle Control'
    perf_table_maxControl = 'Higest Level of Control Achieved'
    mse_barricade = '<para><font fontname="MontserratBold" color="darkred" size="14">Barricade</font></para>'
    mse_slalom = '<para><font fontname="MontserratBold" color="darkred" size="14">Slalom</font></para>'
    mse_reverse = '<para><font fontname="MontserratBold" color="darkred" size="14">Reverse</font></para>'
    mse_ova = '<para><font fontname="MontserratBold" color="darkred" size="14">Overall Performance</font></para>'
    mse_graphTitle = 'Performance Graph'
    mseTable_headers = [['Stress', 'Final Time', 'Reverse', 'Slalom', 'Barricade', Paragraph('<para align="center">Cone<br/> Penalties</para>'), Paragraph('<para align="center">Gate<br/> Penalties</para>'), Paragraph('<para align="center">Final<br/> Result</para>')]]
    gpavg_lvl = 'Group Average'
    
'-----------------------------------------------------------------------------'
'Functions'
'-----------------------------------------------------------------------------'

def createCover(canvas, doc):
#     page_width, page_height = canvas._pagesize
    image = pImage.open(logo_int)
    image_width, image_height = image.size
#     draw_width, draw_height = page_width, page_height

#     canvas.drawImage(logo_int,
#                      pwidth/2, 500, #width=draw_width, height=draw_height,
#                      preserveAspectRatio=True)
    canvas.drawImage(logo_int, ((pwidth/2)-(image_width/2)),pheight-image_height*1.5, mask='auto')
    
def create_deco():
    global header_content, footer_content
    '-----------------------------------------------------------------------------'
    'Header'
    '-----------------------------------------------------------------------------'
    Header_caption=((header_center_text))
    A='Author'
    Header_table_data=[[Image(ceo_cfo_inv, width=230, height=73)]
    ]
    
    Header_table=Table(Header_table_data,
                        colWidths=[pwidth*.84],
                        rowHeights=[3*cm],
                        style=[
    # ('GRID',(0,0),(1,2),1,colors.black),
    # #('ALIGN',(0,0),(1,0),'CENTER'),
    ('ALIGN',(0,0),(0,0),'RIGHT'),
    # ('VALIGN',(0,0),(3,2),'MIDDLE'),
    # ('ALIGN',(2,0),(3,2),'RIGHT'),
    # ('SPAN',(0,0),(0,2)),
    # ('SPAN',(1,0),(1,2)),
    # ('BOX',(2,0),(-1,-1),1,colors.black),
    #('TEXTCOLOR',(0,0),(3,0),colors.Color(49/255,71/255,137/255))
    ])
    '-----------------------------------------------------------------------------'
    'Footer'
    '-----------------------------------------------------------------------------'

    Footer_table_data=[[Image(as3_logo, 
                        width=144, 
                        height=60),
                        Paragraph('<para fontname="MontserratBold" size="12" color="black">WWW.</para><para color="#C10230">AS3</para><para color="black">INTERNATIONAL.COM</para>')]
                        ]
    Footer_table=Table(Footer_table_data,colWidths=[(pwidth*.84)/2,(pwidth*.84)/2],rowHeights=[1*cm] ,style=[
    # ('GRID',(0,0),(-1,-1),1,colors.black),
    #('ALIGN',(0,0),(1,0),'CENTER'),
    ('ALIGN',(0,1),(-1,-1),'RIGHT'),
    ('VALIGN',(0,0),(0,0),'MIDDLE'),
    # ('ALIGN',(0,0),(3,0),'CENTER'),
    #('TEXTCOLOR',(0,0),(3,0),colors.Color(49/255,71/255,137/255))
    ])
    
    footer_content = Footer_table
    header_content = Header_table

create_deco() 

'Image Aspect Ratio'

def get_image(path, width=1*cm):
    img = utils.ImageReader(path)
    iw, ih = img.getSize()
    aspect = ih / float(iw)
    return Image(path, width=width, height=(width * aspect))

def altBackground(data, table):
    data_len = len(data)+1

    for each in range(data_len):
        if each % 2 == 0:
            bg_color = colors.white
        else:
            bg_color = colors.lightgrey

        table.setStyle(TableStyle([('BACKGROUND', (0, each), (-1, each), bg_color)]))

'-----------------------------------------------------------------------------'
'Build story'
'-----------------------------------------------------------------------------'

story = []

'-----------------------------------------------------------------------------'
'Intro Page'
'-----------------------------------------------------------------------------'
story.append(Paragraph(mainTitle, ttl))
story.append(Spacer(0,20))
story.append(Paragraph(student, h2))
story.append(Paragraph(company, no_spacing))
story.append(Paragraph(program, no_spacing))
story.append(NextPageTemplate('NormalPage'))
story.append(PageBreak())

'- Page 2 -'
intro_table_data = [[Paragraph(stdt_name), student],
                    [Paragraph(company_label), company],
                    [Paragraph(pgm_label), program],
                    [Paragraph(pgmDt_label), fulldate],
                    [Paragraph(vhcl_label), vehicle]
                   ]

intro_table = Table(intro_table_data, colWidths=[120,200], style=introTable)

story.append(Spacer(0,20))
story.append(Paragraph(scndTitle, ttl))
story.append(Spacer(0,20))
story.append(intro_table)
story.append(Spacer(0,20))
story.append(Paragraph(intro_title, h1))
story.append(Spacer(0,20))
story.append(Paragraph(intro_text, pstyle))
story.append(PageBreak())

'- Page 3 -'
story.append(Paragraph(thirdTitle, h1))
story.append(Paragraph(thirdSubTitle, h2))
story.append(Spacer(0,50))
story.append(Paragraph(slalomTitle, subh2))
story.append(Spacer(0,20))

slalomGraph = Image('plots/Slalom Graph ' + student + '.png', width=pwidth-inch)
# story.append(slalomGraph)
story.append(get_image('plots/Slalom Graph ' + student + '.png', width=pwidth-inch))
story.append(Paragraph(slalom_text, style=footertxt))
story.append(Spacer(0,80))
story.append(Paragraph(perf_table_title, h3))
'------------------------------------------------------------------------------------'
slalom_table_data = [['','',perf_table_heading],
                     [perf_table_completed, snor, gasnor],
                     [perf_table_runs, spoce, gaspoce],
                     [perf_table_percetage, saoep, gasaoep],
                     [perf_table_control, saovc, gasaovc],
                     [perf_table_maxControl, '', str(sfpl)+'%']
                    ]
slalom_table = Table(slalom_table_data, colWidths=[pwidth*.40, pwidth*.20, pwidth*.20], style=normalTable)
altBackground(slalom_table_data, slalom_table)
'------------------------------------------------------------------------------------'
story.append(slalom_table)
story.append(PageBreak())

'- Page 4 -'
story.append(Spacer(0,50))
story.append(Paragraph(LnChTitle, subh2))
story.append(Spacer(0,20))

slalomGraph = Image('plots/LnCh Graph ' + student + '.png', width=pwidth-inch)
# story.append(slalomGraph)
story.append(get_image('plots/LnCh Graph ' + student + '.png', width=pwidth-inch))
story.append(Paragraph(slalom_text, style=footertxt))
story.append(Spacer(0,80))
story.append(Paragraph(perf_table_title, h3))
'------------------------------------------------------------------------------------'
lnch_table_data = [['','',perf_table_heading],
                     [perf_table_completed, lnor, galnor],
                     [perf_table_runs, lpoce, galpoce],
                     [perf_table_percetage, laoep, galaoep],
                     [perf_table_control, laovc, galaovc],
                     [perf_table_maxControl, '', str(lfpl)+'%']
                    ]
lnch_table = Table(lnch_table_data, colWidths=[pwidth*.40, pwidth*.20, pwidth*.20], style=normalTable)
altBackground(lnch_table_data, lnch_table)
'------------------------------------------------------------------------------------'


story.append(lnch_table)
story.append(PageBreak())

'- Page 5 -'

story.append(Paragraph(fourthSubTitle, h1))
story.append(Spacer(0,20))
story.append(Paragraph(mse_desc))
story.append(Spacer(0,20))
story.append(Paragraph(mse_consoleTitle, h3))
story.append(Spacer(0,10))

'------------------------------------------------------------------------------------'
mse_lnch_graph = get_image('plots/final_lnch_percent-' + student + '.png', width=fwidth*.20)
mse_slalom_graph = get_image('plots/final_slalom_percent-' + student + '.png', width=fwidth*.20)
mse_reverse_graph = get_image('plots/final_rv_percent-' + student + '.png', width=fwidth*.20)
mse_ova_graph = get_image('plots/final_percent-' + student + '.png', width=fwidth*.30)


mse_table_data = [[Paragraph(mse_barricade), Paragraph(mse_slalom), Paragraph(mse_reverse), Paragraph(mse_ova)], 
                  [mse_lnch_graph, mse_slalom_graph, mse_reverse_graph, mse_ova_graph]
                 ]
mse_table = Table(mse_table_data, colWidths=[fwidth*.23, fwidth*.23, fwidth*.23, fwidth*.34], style=consoleTable)
'------------------------------------------------------------------------------------'

story.append(mse_table)
story.append(Spacer(0,5))
story.append(Paragraph(mse_text))
story.append(PageBreak())

'- Page 6 -'
msei = get_image('plots/final_exercise-' + student + '.png', width=fwidth)
msei_data = [[msei]]

# story.append(get_image('plots/final_exercise-' + student + '.png', width=fwidth))
story.append(Spacer(0,20))

'------------------------------------------------------------------------------------'
mse_hist_last_line = [[Paragraph(gpavg_lvl), mseg_t, str(mseg_rev_pc)+'%', '','', mseg_c, mseg_g, str(mseg_perf)+'%']]
mse_hist_data = mseTable_headers + np.array(mse_data.values).tolist() + np.array(mse_hist_last_line).tolist()

mse_table = Table(mse_hist_data, colWidths=[pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1], style=mseTable)

story.append(Paragraph(mse_graphTitle, h3))
story.append(Spacer(0,10))
story.append(mse_table)
story.append(Spacer(0,20))
story.append(Table(msei_data, colWidths=None, rowHeights=None, style=consoleTable))
story.append(Spacer(0,20))
story.append(Paragraph(inst_comments_title, h3))
story.append(Spacer(0,20))
story.append(Paragraph(comment))
story.append(Spacer(0,20))
story.append(Paragraph(fn_note))

'----------------------------------------------------------------'
'Document Creation'
'----------------------------------------------------------------'

doc = MyDocTemplate('AS3_Student_Report '+ company + '_Sample.pdf', pagesize=letter)
doc.multiBuild(story)

1